In [1]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

import re
import pickle
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, make_scorer
from vowpalwabbit.sklearn_vw import VWClassifier, VW
import itertools
from sklearn.decomposition import NMF, TruncatedSVD

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse import csr_matrix, hstack
import imblearn
from glob import glob

In [33]:
def sparsematrix(X):
    row = []
    col = []
    data = []
    for r in range(X.shape[0]):
        row_counter = Counter(X[r])
        for site, num in row_counter.items():
            row.append(r)
            col.append(site)
            data.append(num)
    print "Sparse Matrix - rows:", X.shape[0], "columns:", len(set(col))
    return csr_matrix((data, (row, col)), shape=(X.shape[0], len(set(col))))[:,1:]


def sites_to_sparse_tfidf(train_data, test_data, target_col, session_length, label_encoder=False):
    train_test_df = pd.concat([train_data, test_data])
    train_index_full = list(train_data.index)
    train_index_dup = list(train_data[train_data.duplicated(subset=['site' + str(c) for c in range(1,session_length+1)], keep=False)]\
                           [['site' + str(c) for c in range(1,10+1)]+["target"]].index)
    test_index_full = list(test_data.index)
    test_index_dup = list(test_data[test_data.duplicated(subset=['site' + str(c) for c in range(1,session_length+1)], keep=False)]\
                           [['site' + str(c) for c in range(1,10+1)]].index)
    train_duplicates_mask = np.transpose([np.in1d(train_index_full, train_index_dup).astype(int)])
    test_duplicates_mask = np.transpose([np.in1d(test_index_full, test_index_dup).astype(int)])

    y = train_data[target_col]

    train_test_df_sites = train_test_df[['site' + str(c) for c in range(1,10+1)]].fillna(0).astype('int')
    train_test_df_sites_array = [" ".join(["s_"+str(s) for s in train_test_df_sites.as_matrix()[i] if int(s) != 0]) \
                                                                  for i in range(train_test_df_sites.shape[0])]

    tfidf = TfidfVectorizer(max_df=0.9).fit(train_test_df_sites_array) #TfidfVectorizer()
    X_train_test_sparse = tfidf.transform(train_test_df_sites_array)

    X_train_sparse = X_train_test_sparse[:len(train_data)]
    X_test_sparse = X_train_test_sparse[len(train_data):]
    
    sites_columns_num = X_train_test_sparse.shape[1]
    
    y_for_vw = None
    class_encoder = None
    if label_encoder:
        class_encoder = LabelEncoder().fit(y.astype('str'))
        y_for_vw = class_encoder.transform(y.astype('str')) + 1
    
    return [X_train_sparse, X_test_sparse, y, y_for_vw, sites_columns_num, class_encoder, tfidf, \
             train_duplicates_mask, test_duplicates_mask]


def features_to_sparse(train_data, test_data, feature_cols):
    features_matrix = []
    for df in [train_data, test_data]:
        num_cols = 0
        data = []
        rows = []
        cols = []
        for label in feature_cols:
            if label in ["day_of_week", "daytime"]:
                coldata = list(df[[label]].values.T[0].astype('float') + 1)
            else:
                coldata = list(df[[label]].values.T[0].astype('float'))
            if len(data):
                data += coldata
            else:
                data = list(coldata)
            if len(cols):
                cols += [num_cols] * len(coldata)
            else:
                cols = [num_cols] * len(coldata)
            num_cols += 1
        rows = [r for r in range(df.shape[0])] * num_cols
        features = csr_matrix((data, (rows, cols)), shape=(df.shape[0], num_cols), dtype=float)
        features_matrix.append(features)
    return features_matrix


def calc_site_times_portions(train_data, test_data):
    site_times = [{},{}]
    count = 0
    for data in [train_data, test_data]:
        for r, row in data[:][range(0, 10)+range(20,30)].iterrows():
            rowdic = {}
            for c, s in [[c, 'site' + str(c)] for c in range(1,10)]:
                if row[s] == 0:
                    continue
                if row[s] in rowdic:
                    rowdic[int(row[s])] += row["time_diff"+str(c)]
                else:
                    rowdic[int(row[s])] = row["time_diff"+str(c)]
            site_times[count][r] = {}
            for site, time in rowdic.items():
                if len(rowdic) == 1:
                    site_times[count][r][int(site)] = 1.0
                    #site_times[count][r][int(site)] = 30*60
                    continue
                if time > 0:
                    site_times[count][r][int(site)] = round(float(time)/row["session_timespan"],5)
                    #site_times[count][r][int(site)] = int(time)
        count+=1
    return site_times

def calc_site_user_relevance(train_data, test_data, site_dict):
    site_rels = [{}, {}]
    count = 0
    for data in [train_data, test_data]:
        for r, row in data[:][range(0, 10)].iterrows():
            rowdic = {}
            notinsitedict = 0
            for c, s in [[c, 'site' + str(c)] for c in range(1,11)]:
                if row[s] == 0:
                    continue
                if row[s] not in rowdic:
                    if int(row[s]) in site_dict:
                        rowdic[int(row[s])] = round(1./len(site_dict[int(row[s])]), 5)
                    else:
                        notinsitedict += 1
                        rowdic[int(row[s])] = 0
            if len(rowdic) > 0:
                rowdic.update((x, y/(notinsitedict+1)) for x, y in rowdic.items())
                site_rels[count][r] = dict(rowdic)
            else:
                site_rels[count][r] = {}
        count+=1
    return site_rels

def site_rels_to_sparse(siterels):
    row = []
    col = []
    data = []
    rowcount = 0
    for siterel in siterels:
        for r, sites in sorted(siterel.items()):
            if len(sites) < 1:
                continue
            for site, p in sites.items():
                col.append(site)
                row.append(rowcount)
                data.append(p)
            rowcount+=1
    site_times_sparse = csr_matrix((data, (row, col)), shape=(len(siterels[0])+len(siterels[1]), max(col)+1), \
                                                                                              dtype=float)[:,1:]
    return site_times_sparse    
    
    
def site_times_to_sparse(sitetimes):
    row = []
    col = []
    data = []
    rowcount = 0
    for sitetime in sitetimes:
        for r, sites in sorted(sitetime.items()):
            for site, p in sites.items():
                col.append(site)
                row.append(rowcount)
                data.append(p)
            rowcount+=1
    site_times_sparse = csr_matrix((data, (row, col)), shape=(len(sitetimes[0])+len(sitetimes[1]), max(col)+1), \
                                                                                              dtype=float)[:,1:]
    return site_times_sparse


def combine_sites_features_sparse(sites_train_sparse, sites_seq_train_sparse, features_train_sparse, \
                                  sites_test_sparse, sites_seq_test_sparse, features_test_sparse, test_preds_sparse=None,\
                                  train_site_times_sparse = None, test_site_times_sparse = None, \
                                  train_site_rels_sparse = None, test_site_rels_sparse = None, \
                                train_sites_sequence=None, test_sites_sequence=None):
    if train_site_times_sparse is not None and test_site_times_sparse is not None:
        X_train_sparse = hstack([sites_train_sparse, features_train_sparse, train_preds_sparse,\
                                 train_site_times_sparse, train_site_rels_sparse], dtype=float).tocsr()
        X_test_sparse = hstack([sites_test_sparse, features_test_sparse, test_preds_sparse,\
                                test_site_times_sparse, test_site_rels_sparse], dtype=float).tocsr()
    else:
        X_train_sparse = hstack([sites_train_sparse, sites_seq_train_sparse], dtype=float).tocsr()
        X_test_sparse = hstack([sites_test_sparse, sites_seq_test_sparse], dtype=float).tocsr()
        
    #X_train_sparse = hstack([X_train_sparse, train_duplicates_mask], dtype=float).tocsr()
    #X_test_sparse = hstack([X_test_sparse, test_duplicates_mask], dtype=float).tocsr() 
    return [X_train_sparse, X_test_sparse]


def sparse_matrix_to_vw(X_sparse, sites_columns_num, sites_seq_columns_num, y=None, weights=None, mark_duplicates=False, mycolumns=[]):
    sessions = {}
    used = {}
    prediction = {}
    day_of_week = {}
    start_hour = {}
    daytime = {}
    unique_sites = {}
    top30_portion = {}
    fb_portion = {}
    youtube_portion = {}
    bot30_portion = {}
    site_longest_time = {}
    session_timespan = {}
    sitetimes = {}
    siterels = {}
    sequence = {}
    
    lables = {}
    lable_weights = {}
    
    #X_sparse = X_sparse_full[:,:-1]
    
    add_features = False
    for r, c in zip(X_sparse.nonzero()[0], X_sparse.nonzero()[1]):
        if tuple([r,c]) not in used:
            used[tuple([r, c])] = 1
            if add_features:
                #if c >= X_sparse.shape[1] - 2 * sites_columns_num - 550 and \
                    #c < X_sparse.shape[1] - 2 * sites_columns_num:
                    #sites_length = X_sparse.shape[1] - 2 * sites_columns_num - len(mycolumns) - 550
                    #if r not in prediction:
                        #prediction[r] = " |aprediction {}:{}".format(int(c - sites_length - len(mycolumns) + 1), int(X_sparse[r,c]))
                    #else:
                        #prediction[r] += " {}:{}".format(int(c - sites_length - len(mycolumns) + 1), int(X_sparse[r,c]))
                    #prediction[r] = " |prediction:100 {}".format(int(X_sparse[r,c]))
                    #continue
                if c == X_sparse.shape[1] - len(mycolumns) + mycolumns.index("prediction"):
                    prediction[r] = " |aprediction {}:{}".format(int(X_sparse[r,c]), 100)
                    continue
                if c == X_sparse.shape[1] - len(mycolumns) + mycolumns.index("day_of_week"): #- 2 * sites_columns_num - 550:
                    day_of_week[r] = " |bday_of_week {}".format(int(X_sparse[r,c]))
                    #day_of_week[r] = " day_of_week:{}".format(int(X_sparse[r,c]))
                    continue
                elif c == X_sparse.shape[1] - len(mycolumns) + mycolumns.index("start_hour"): # - 2 * sites_columns_num  - 550:
                    start_hour[r] = " |chour_start {}".format(int(X_sparse[r,c]))
                    #start_hour[r] = " start_hour:{}".format(int(X_sparse[r,c]))
                    continue
                elif c == X_sparse.shape[1] - len(mycolumns) + mycolumns.index("daytime"): # - 2 * sites_columns_num  - 550:
                    daytime[r] = " |dtime_of_day {}".format(int(X_sparse[r,c]))
                    #daytime[r] = " daytime:{}".format(int(X_sparse[r,c]))
                    continue
                elif c == X_sparse.shape[1] - len(mycolumns) + mycolumns.index("session_timespan"): # - 2 * sites_columns_num  - 550:
                    session_timespan[r] = " |jsession_timespan time:{}".format(int(X_sparse[r,c]))
                    #session_timespan[r] = " session_timespan:{}".format(int(X_sparse[r,c]))
                    continue
                elif c == X_sparse.shape[1] - len(mycolumns) + mycolumns.index("#unique_sites"): # - 2 * sites_columns_num - 550:
                    unique_sites[r] = " unique_sites:{}".format(int(X_sparse[r,c]))
                    #unique_sites[r] = " unique_sites:{}".format(X_sparse[r,c])
                    continue
                elif c == X_sparse.shape[1] - len(mycolumns) + mycolumns.index("site_longest_time"): # - 2 * sites_columns_num - 550:
                    site_longest_time[r] = " |hsite_longest_time {}:{}".format(int(X_sparse[r,c]), 3)
                    #site_longest_time[r] = " site_longest_time:{}".format(int(X_sparse[r,c]))
                    continue
                elif c == X_sparse.shape[1] - len(mycolumns) + mycolumns.index("top30_portion"): # - 2 * sites_columns_num - 550:
                    top30_portion[r] = " top30:{}".format(X_sparse[r,c])
                    continue
                elif c == X_sparse.shape[1] - len(mycolumns) + mycolumns.index("bot30_portion"): # - 2 * sites_columns_num - 550:
                    bot30_portion[r] = " bot30:{}".format(X_sparse[r,c])
                    continue
                elif c == X_sparse.shape[1] - len(mycolumns) + mycolumns.index("fb_portion"): # - 2 * sites_columns_num - 550:
                    fb_portion[r] = " facebook:{}".format(X_sparse[r,c])
                    continue
                elif c == X_sparse.shape[1] - len(mycolumns) + mycolumns.index("youtube_portion"): # - 2 * sites_columns_num - 550:
                    youtube_portion[r] = " youtube:{}".format(X_sparse[r,c])
                    continue
                    
            if c < sites_columns_num: #X_sparse.shape[1] - len(mycolumns): #
                if r in sessions:
                    sessions[r] += " {}:{}".format(int(c+1), X_sparse[r,c])
                else:
                    if y is not None:
                        sessions[r] = ' |site' + " {}:{}".format(int(c+1), X_sparse[r,c])
                        lables[r] = str(y[r])
                        if weights is not None:
                            lable_weights[r] = str(weights[y[r]-1])
                    else:
                        sessions[r] = ' |site' + " {}:{}".format(int(c+1), X_sparse[r,c])
            elif c >= sites_columns_num and c < sites_columns_num + sites_seq_columns_num:
                if r in sequence:
                    sequence[r] += " {}:{}".format(int(c+1), X_sparse[r,c])
                else:
                    sequence[r] = ' |zsequence' + " {}:{}".format(int(c+1), X_sparse[r,c])
            #elif c >= X_sparse.shape[1] - 2 * sites_columns_num and c < X_sparse.shape[1] - sites_columns_num:
                #if r in sitetimes:
                    #sitetimes[r] += " {}:{}".format(int(c - sites_columns_num - len(mycolumns)+1 - 550), float(X_sparse[r,c]))
                #else:
                    #sitetimes[r] = ' |isitetime' + " {}:{}".format(int(c - sites_columns_num - len(mycolumns)+1 - 550), float(X_sparse[r,c]))
            #elif c >= X_sparse.shape[1] - sites_columns_num:
                #if r in siterels:
                    #siterels[r] += " {}:{}".format(int(c - 2 * sites_columns_num - len(mycolumns)+1 - 550), float(X_sparse[r,c]))
                #else:
                    #siterels[r] = ' |lsiterels' + " {}:{}".format(int(c - 2 * sites_columns_num - len(mycolumns)+1 - 550), float(X_sparse[r,c]))
        
    
    return {"sites": sessions, "lables": lables, "lable_weights": lable_weights, "prediction": prediction, "day_of_week": day_of_week, \
                      "start_hour": start_hour, "daytime": daytime, \
                     "unique_site": unique_sites, "top30_portion": top30_portion, \
                    "bot30_portion": bot30_portion, "fb_portion": fb_portion, \
                    "youtube_portion": youtube_portion, "site_longest_time": site_longest_time, \
                    "session_timespan": session_timespan, "sitetimes": sitetimes, "siterels": siterels, "sequence": sequence}



def vw_to_file(sites, out_file, features={}, lables={}, lable_weights={},  quiet=True):   
    vw_writer = open(out_file, 'w')
    final_vw = {}
    gen_features = []
    
    if not quiet:
        print "Features:", features.keys()
        
    for r in sorted(sites.keys()):
        if r in lables:
            final_vw[r] = lables[r]
        else:
            final_vw[r] = ""
        if r in lable_weights:
            final_vw[r] += " {}".format(lable_weights[r])
        final_vw[r] += sites[r] #+ " |features"
        for fname, feature in features.items():
            if fname in ["youtube_portion", "fb_portion", "top30_portion", "bot30_portion", \
                                         "unique_sites"] and r in feature:
                gen_features.append(feature[r])
                continue
            if r in feature:
                final_vw[r] += feature[r]        
            
        if len(gen_features):
            final_vw[r] += " |features"
            for gf in gen_features:
                final_vw[r] += gf
        gen_features = []
        
        #if "prediction" in features and r in features["prediction"]:
            #final_vw[r] += features["prediction"][r]
        
        vw_writer.write(final_vw[r] + "\n")
        
    vw_writer.close()
    
    
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)



In [5]:
def calc_predictions(train_data, test_data):
    test_row_users = {}
    train_row_users = {}
    
    # Add predictions from the dataframe (based on uniquely visited site)
    for r, v in test_data[["prediction"]].iterrows():
        if int(v) != 0:
            test_row_users[r] = [int(v)]
    
    
    #Identify sessions with identical sites sequence
    train_index_full = list(train_data.index)
    train_index_dup = list(train_data[train_data.duplicated(subset=['site' + str(c) for c in range(1,10+1)], keep=False)]\
                           [['site' + str(c) for c in range(1,10+1)]+["target"]].index)

    test_index_full = list(test_data.index)
    test_index_dup = list(test_data[test_data.duplicated(subset=['site' + str(c) for c in range(1,10+1)], keep=False)]\
                           [['site' + str(c) for c in range(1,10+1)]].index)
    
    train_user_dup_rows_dict = {}
    train_dup_row_users_dict = {}

    test_dup_rows_dict = {}
    

    sites_cols = ['site' + str(c) for c in range(1,10+1)]
    
    for r, row in train_data.iloc[train_index_dup][sites_cols+["target"]].iterrows():
        if row["target"] in train_user_dup_rows_dict:
            if tuple(row[sites_cols]) in train_user_dup_rows_dict[row["target"]]:
                train_user_dup_rows_dict[row["target"]][tuple(row[sites_cols])] += 1
            else:
                train_user_dup_rows_dict[row["target"]][tuple(row[sites_cols])] = 1 
        else:
            train_user_dup_rows_dict[row["target"]] = {tuple(row[sites_cols]): 1}

        if tuple(row[sites_cols]) in train_dup_row_users_dict:
            train_dup_row_users_dict[tuple(row[sites_cols])].add(row["target"])
        else:
            train_dup_row_users_dict[tuple(row[sites_cols])] = set([row["target"]])

    for r, row in test_data.iloc[test_index_dup][sites_cols].iterrows():  
        if tuple(row[sites_cols]) in test_dup_rows_dict:
            test_dup_rows_dict[tuple(row[sites_cols])] += 1
        else:
            test_dup_rows_dict[tuple(row[sites_cols])] = 1

        if tuple(row[sites_cols]) in train_dup_row_users_dict:
            if r in test_row_users:
                pass #don't overwright predictions from the dataframe
                #test_row_users[r] += train_dup_row_users_dict[tuple(row[sites_cols])]
            else:
                test_row_users[r] = train_dup_row_users_dict[tuple(row[sites_cols])]
        
    # Find users who visited 2 websites
    site_pairs = {}
    for r, row in train_data[sites_cols+["target"]].iterrows():
        unique_sites = Counter(row).keys()
        if 0 in unique_sites:
            del unique_sites[unique_sites.index(0)]
        if len(unique_sites) > 1:
            for subset in itertools.permutations(Counter(row).keys(), 2):
                if tuple(subset) in site_pairs:
                    site_pairs[tuple(subset)].add(row["target"])
                else:
                    site_pairs[tuple(subset)] = set([row["target"]])
        if len(unique_sites) > 2:
            for subset in itertools.permutations(Counter(row).keys(), 3):
                if tuple(subset) in site_pairs:
                    site_pairs[tuple(subset)].add(row["target"])
                else:
                    site_pairs[tuple(subset)] = set([row["target"]])
        if len(unique_sites) > 3:
            for subset in itertools.permutations(Counter(row).keys(), 4):
                if tuple(subset) in site_pairs:
                    site_pairs[tuple(subset)].add(row["target"])
                else:
                    site_pairs[tuple(subset)] = set([row["target"]])
    
    
    # Add predictions to test data based on 2 visited websites
    for r, row in test_data[sites_cols].iterrows():
        unique_sites = Counter(row).keys()
        if len(unique_sites) > 1:
            for subset in itertools.permutations(Counter(row).keys(), 2):
                if subset in site_pairs:
                    if r in test_row_users:
                        pass
                    else:
                        test_row_users[r] = list(site_pairs[subset])
        if len(unique_sites) > 2:
            for subset in itertools.permutations(Counter(row).keys(), 3):
                if subset in site_pairs:
                    if r in test_row_users:
                        pass
                    else:
                        test_row_users[r] = list(site_pairs[subset])
        if len(unique_sites) > 3:
            for subset in itertools.permutations(Counter(row).keys(), 4):
                if subset in site_pairs:
                    if r in test_row_users:
                        pass
                    else:
                        test_row_users[r] = list(site_pairs[subset])
        
    
    
    return test_row_users, site_pairs

In [6]:
def calc_predictions(train_data, test_data, site_dic, user_dic, min_users, max_users, permutations=False):
    train_row_users = {}
    test_row_users = {}
    
    sites_cols = ['site' + str(c) for c in range(1,10+1)]
    
    # Add predictions from the dataframe (based on uniquely visited site)
    for r, v in train_data[["prediction"]].iterrows():
        if int(v) != 0:
            train_row_users[r] = {int(v): 1}  
    
    for r, v in test_data[["prediction"]].iterrows():
        if int(v) != 0:
            test_row_users[r] = {int(v): 1}
    
    # Add predictions if a website in session was visited by less than num_users_for_prediction
    for r, row in train_data[sites_cols+["target"]].iterrows():
        if r in train_row_users:
            continue
        session_predictions = {}
        for site in row:
            predictions = set([])
            if site in site_dic and site in user_dic[int(row["target"])] \
                          and len(site_dic[site]) in range(min_users, max_users+1):
                predictions = set(site_dic[site])
            if len(predictions):
                for puser in predictions:
                    if puser in session_predictions:
                        session_predictions[puser] +=1
                    else:
                        session_predictions[puser] = 1
                #session_predictions |= predictions
        if len(session_predictions):
            train_row_users[r] = session_predictions
    
    
    for r, row in test_data[sites_cols].iterrows():
        if r in test_row_users:
            continue
        session_predictions = {}
        for site in row:
            predictions = set([])
            if site in site_dic and len(site_dic[site]) in range(min_users, max_users):
                predictions = set(site_dic[site])
            if len(predictions):
                for puser in predictions:
                    if puser in session_predictions:
                        session_predictions[puser] +=1
                    else:
                        session_predictions[puser] = 1
                #session_predictions |= predictions
        if len(session_predictions):
            test_row_users[r] = session_predictions
    
    if not permutations:
        return train_row_users, test_row_users
    
    #Identify sessions with identical sites sequence
    train_index_full = list(train_data.index)
    train_index_dup = list(train_data[train_data.duplicated(subset=['site' + str(c) for c in range(1,10+1)], keep=False)]\
                           [['site' + str(c) for c in range(1,10+1)]+["target"]].index)

    test_index_full = list(test_data.index)
    test_index_dup = list(test_data[test_data.duplicated(subset=['site' + str(c) for c in range(1,10+1)], keep=False)]\
                           [['site' + str(c) for c in range(1,10+1)]].index)
    
    train_user_dup_rows_dict = {}
    train_dup_row_users_dict = {}

    #test_dup_rows_dict = {} 

    
    
    for r, row in train_data.ix[train_index_dup][sites_cols+["target"]].iterrows():
        if row["target"] in train_user_dup_rows_dict:
            if tuple(row[sites_cols]) in train_user_dup_rows_dict[row["target"]]:
                train_user_dup_rows_dict[row["target"]][tuple(row[sites_cols])] += 1
            else:
                train_user_dup_rows_dict[row["target"]][tuple(row[sites_cols])] = 1 
        else:
            train_user_dup_rows_dict[row["target"]] = {tuple(row[sites_cols]): 1}

        if tuple(row[sites_cols]) in train_dup_row_users_dict:
            train_dup_row_users_dict[tuple(row[sites_cols])].add(row["target"])
        else:
            train_dup_row_users_dict[tuple(row[sites_cols])] = set([row["target"]])
    
    # Make predictions based on duplicate sessions
    for r, row in train_data.ix[train_index_dup][sites_cols].iterrows():        
        if tuple(row[sites_cols]) in train_dup_row_users_dict:
            if r in train_row_users:
                pass #don't overwright predictions from the dataframe
                #train_row_users[r] += train_dup_row_users_dict[tuple(row[sites_cols])]
            else:
                train_row_users[r] = train_dup_row_users_dict[tuple(row[sites_cols])]
    
    for r, row in test_data.ix[test_index_dup][sites_cols].iterrows():  
        #if tuple(row[sites_cols]) in test_dup_rows_dict:
            #test_dup_rows_dict[tuple(row[sites_cols])] += 1
        #else:
            #test_dup_rows_dict[tuple(row[sites_cols])] = 1

        if tuple(row[sites_cols]) in train_dup_row_users_dict:
            if r in test_row_users:
                pass #don't overwright predictions from the dataframe
                #test_row_users[r] += train_dup_row_users_dict[tuple(row[sites_cols])]
            else:
                test_row_users[r] = train_dup_row_users_dict[tuple(row[sites_cols])]
    

    
    
    # Find users who visited 2, 3, 4 websites
    site_pairs = {}
    for r, row in train_data[sites_cols+["target"]].iterrows():
        unique_sites = Counter(row).keys()
        if 0 in unique_sites:
            del unique_sites[unique_sites.index(0)]
        if len(unique_sites) > 1:
            for subset in itertools.permutations(Counter(row).keys(), 2):
                if tuple(subset) in site_pairs:
                    site_pairs[tuple(subset)].add(row["target"])
                else:
                    site_pairs[tuple(subset)] = set([row["target"]])
        if len(unique_sites) > 2:
            for subset in itertools.permutations(Counter(row).keys(), 3):
                if tuple(subset) in site_pairs:
                    site_pairs[tuple(subset)].add(row["target"])
                else:
                    site_pairs[tuple(subset)] = set([row["target"]])
        #if len(unique_sites) > 3:
            #for subset in itertools.permutations(Counter(row).keys(), 4):
                #if tuple(subset) in site_pairs:
                    #site_pairs[tuple(subset)].add(row["target"])
                #else:
                    #site_pairs[tuple(subset)] = set([row["target"]])
    
    # Add predictions to train data based on 2 visited websites
    for r, row in train_data[sites_cols+["target"]].iterrows():
        if r in train_row_users:
            continue
        unique_sites = Counter(row).keys()
        if 0 in unique_sites:
            del unique_sites[unique_sites.index(0)]
        if len(unique_sites) > 1:
            for subset in itertools.permutations(Counter(row).keys(), 2):
                if tuple(subset) in site_pairs:
                    if r in train_row_users:
                        train_row_users[r] |= site_pairs[subset]
                    else:
                        train_row_users[r] = set(site_pairs[subset])
        if len(unique_sites) > 2:
            for subset in itertools.permutations(Counter(row).keys(), 3):
                if tuple(subset) in site_pairs:
                    if r in test_row_users:
                        train_row_users[r] |= site_pairs[subset]
                    else:
                        train_row_users[r] = set(site_pairs[subset])
        #if len(unique_sites) > 3:
            #for subset in itertools.permutations(Counter(row).keys(), 4):
                #if tuple(subset) in site_pairs:
                    #if r in test_row_users:
                        #train_row_users[r].add(site_pairs[subset])
                    #else:
                        #train_row_users[r] = set(site_pairs[subset])
    
    # Add predictions to test data based on 2 visited websites
    for r, row in test_data[sites_cols].iterrows():
        if r in test_row_users:
            continue
        unique_sites = Counter(row).keys()
        if len(unique_sites) > 1:
            for subset in itertools.permutations(Counter(row).keys(), 2):
                if subset in site_pairs:
                    if r in test_row_users:
                        test_row_users[r] |= site_pairs[subset]
                    else:
                        test_row_users[r] = set(site_pairs[subset])
        if len(unique_sites) > 2:
            for subset in itertools.permutations(Counter(row).keys(), 3):
                if subset in site_pairs:
                    if r in test_row_users:
                        test_row_users[r] |= site_pairs[subset]
                    else:
                        test_row_users[r] = set(site_pairs[subset])
        #if len(unique_sites) > 3:
            #for subset in itertools.permutations(Counter(row).keys(), 4):
                #if subset in site_pairs:
                    #if r in test_row_users:
                        #test_row_users[r].add(site_pairs[subset])
                    #else:
                        #test_row_users[r] = set(site_pairs[subset])
        
    
    
    return train_row_users, test_row_users

In [7]:
def create_user_site_dic(train_data, site_freq_pkl):
    user_dic = {}
    site_dic = {}

    pkl_file = open(site_freq_pkl, 'rb')
    site_freq = pickle.load(pkl_file)
    #top_sites = [v[1] for k, v in sorted(site_freq.items(), key=lambda t: t[1][1], reverse=True)[:0]]
    
    for i, v in train_data.iterrows():
        if v.target not in user_dic:
            user_dic[v.target] = {}
        for site in ['site' + str(i) for i in range(1,11)]:
            if int(v[site]) != 0: #and v[site] not in top_sites:
                if v[site] in user_dic[v.target]:
                    user_dic[v.target][v[site]] +=1
                else:
                    user_dic[v.target][v[site]] = 1

                if v[site] in site_dic:
                    site_dic[v[site]].add(v.target)
                else:
                    site_dic[v[site]] = set([v.target])
    
    return user_dic, site_dic

In [8]:
def text_classifier(vectorizer, transformer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )


# Let's Start

In [9]:
%%time
#1
train_data = pd.read_csv('kaggle_data/train_sessions_sdh.csv', index_col="session_id")
#train_data = pd.read_csv("kaggle_data/full_train_w7.csv")
test_data = pd.read_csv('kaggle_data/test_sessions_sdh.csv', index_col="session_id")

CPU times: user 1.56 s, sys: 92 ms, total: 1.66 s
Wall time: 1.69 s


In [ ]:
%%time
user_dic, site_dic = create_user_site_dic(train_data, "kaggle_data/site_freq.pkl")

KeyboardInterrupt: 

In [12]:
%%time
train_predictions, test_predictions = calc_predictions(train_data, test_data, \
                                                       site_dic, user_dic, 1, 2)

CPU times: user 21.4 s, sys: 104 ms, total: 21.5 s
Wall time: 21.3 s


In [86]:
%%time
train_preds = {}
for k, v in train_predictions.items():
    train_preds[k] = {}
    for user, count in v.items():
        if count not in train_preds[k]:
            train_preds[k][count] = [user]
        else:
            train_preds[k][count].append(user)
    train_preds[k] = train_preds[k][np.max(train_preds[k].keys())]

test_preds = {}
for k, v in test_predictions.items():
    test_preds[k] = {}
    for user, count in v.items():
        if count not in test_preds[k]:
            test_preds[k][count] = [user]
        else:
            test_preds[k][count].append(user)
    test_preds[k] = test_preds[k][np.max(test_preds[k].keys())]

NameError: name 'train_predictions' is not defined

In [14]:
%%time
#2
#train_site_sequence = csr_matrix(train_data[['site' + str(c) for c in range(1,10+1)]].as_matrix(), dtype=int)
#test_site_sequence = csr_matrix(test_data[['site' + str(c) for c in range(1,10+1)]].as_matrix(), dtype=int)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.81 µs


In [15]:
%%time
#3
# Additionally, let's calculate the percentage of session time spent by every site in session.
site_times = calc_site_times_portions(train_data, test_data)

# Relevance of a given site in session to the given user.
site_rels = calc_site_user_relevance(train_data, test_data, site_dic)

CPU times: user 1min 27s, sys: 392 ms, total: 1min 27s
Wall time: 1min 27s


In [16]:
%%time
#4
# Convert site times to sparse format.
site_times_sparse = site_times_to_sparse(site_times)
train_site_times_sparse = site_times_sparse[:len(train_data)]
test_site_times_sparse = site_times_sparse[len(train_data):]

# Convert site relevance to sparse format.
site_rels_sparse = site_rels_to_sparse(site_rels)
train_site_rels_sparse = site_rels_sparse[:len(train_data)]
test_site_rels_sparse = site_rels_sparse[len(train_data):]

CPU times: user 1.16 s, sys: 68 ms, total: 1.22 s
Wall time: 1.2 s


In [11]:
_end = '_end_'
def make_trie(session):
    root = dict()
    current_dict = root
    for site in session:
        current_dict = current_dict.setdefault(site, {})
    current_dict[_end] = _end
    return root

def in_trie(trie, sequence):
    current_dict = trie
    found = False
    num = len(sequence)
    counter = 0
    for site in sequence:
        while _end not in current_dict:
            if site in current_dict:
                found = True
                counter += 1
                current_dict = current_dict[site]
                break
            else:
                found = False
                current_dict = current_dict.itervalues().next()
                if _end in current_dict:
                    return False
    else:
        if found == True and counter == num:
            return True
        else:
            return False

In [48]:
trie = make_trie(train_data[range(0,10)].iloc[14].as_matrix().astype(str))
trie

{'20': {'32': {'662': {'690': {'32': {'690': {'662': {'690': {'662': {'32': {'_end_': '_end_'}}}}}}}}}}}

In [53]:
in_trie(trie, ['32', '32', '690', '20'])

False

In [10]:
%%time
# Let's find frequent patterns is sessions for every user
train_df_sites = train_data[['site' + str(c) for c in range(1,10+1)]+["user_id"]].fillna(0).astype('int')
user_seq_dic = {}
seq_user_dic = {}

for r in range(len(train_df_sites)):
    session = train_df_sites.iloc[r,:-1].as_matrix().astype(int)
    session = np.delete(session, np.where(session.astype(int) == 0)[0])
    sequence = []
    user = int(train_df_sites.iloc[r,-1])

    #if train_df_sites.iloc[r,-1] not in user_seq_dic:
        #user_seq_dic[train_df_sites.iloc[r,-1]] = {}
    #seq_dic = user_seq_dic[train_df_sites.iloc[r,-1]]
    
    ses_seq_set = set()

    it = np.nditer(session, flags=['f_index'])
    while not it.finished:
        ses_seq_set.add(tuple([int(it[0])]))
        asession = session[it.index+1:].copy()
        if len(asession) > 0:      
            ita = np.nditer(asession, flags=['f_index'])
            while not ita.finished:
                ses_seq_set.add(tuple([int(it[0]), int(ita[0])]))
                bsession = asession[ita.index+1:]
                if len(bsession) > 0:
                    itb = np.nditer(bsession, flags=['f_index'])
                    while not itb.finished:
                        ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0])]))
                        csession = bsession[itb.index+1:]
                        if len(csession) > 0:
                            itc = np.nditer(csession, flags=['f_index'])
                            while not itc.finished:
                                ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0])]))
                                dsession = csession[itc.index+1:]
                                if len(dsession) > 0:
                                    itd = np.nditer(dsession, flags=['f_index'])
                                    while not itd.finished:
                                        ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0])]))
                                        #esession = dsession[itd.index+1:]
                                        #if len(esession) > 0:
                                            #ite = np.nditer(esession, flags=['f_index'])
                                            #while not ite.finished:
                                                #ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0]), int(ite[0])]))
                                                #fsession = esession[ite.index+1:]
                                                #if len(fsession) > 0:
                                                    #itf = np.nditer(fsession, flags=['f_index'])
                                                    #while not itf.finished:
                                                        #ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0]), \
                                                                               #int(ite[0]), int(itf[0])]))
                                                        #gsession = fsession[itf.index+1:]
                                                        #if len(gsession) > 0:
                                                            #itg = np.nditer(gsession, flags=['f_index'])
                                                            #while not itg.finished:
                                                                #sequence.append([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0]), \
                                                                #int(ite[0]), int(itf[0]), int(itg[0])])
                                                                #if tuple(sequence[-1]) in seq_dic:
                                                                    #if new_seq: seq_dic[tuple(sequence[-1])] +=1
                                                                #else:
                                                                    #seq_dic[tuple(sequence[-1])] = 1
                                                                #hsession = gsession[itg.index+1:]
                                                                #if len(hsession) > 0:
                                                                    #ith = np.nditer(hsession, flags=['f_index'])
                                                                    #while not ith.finished:
                                                                        #sequence.append([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0]), \
                                                                            #int(ite[0]), int(itf[0]), int(itg[0]), int(ith[0])])
                                                                        #if tuple(sequence[-1]) in seq_dic:
                                                                            #if new_seq: seq_dic[tuple(sequence[-1])] +=1
                                                                        #else:
                                                                            #seq_dic[tuple(sequence[-1])] = 1
                                                                        #isession = hsession[ith.index+1:]
                                                                        #if len(isession) > 0:
                                                                            #iti = np.nditer(isession, flags=['f_index'])
                                                                            #while not iti.finished:
                                                                                #sequence.append([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0]), \
                                                                                        #int(ite[0]), int(itf[0]), int(itg[0]), int(ith[0]), int(iti[0])])
                                                                                #if tuple(sequence[-1]) in seq_dic:
                                                                                    #if new_seq: seq_dic[tuple(sequence[-1])] +=1
                                                                                #else:
                                                                                    #seq_dic[tuple(sequence[-1])] = 1
                                                                                #iti.iternext()
                                                                        #ith.iternext()
                                                                #itg.iternext()
                                                        #itf.iternext()
                                                #ite.iternext()
                                        itd.iternext()
                                itc.iternext()
                        itb.iternext()
                ita.iternext()
        it.iternext()
    
    for seq in ses_seq_set:
        #if seq in seq_dic:
            #seq_dic[seq] +=1
        #else:
            #seq_dic[seq] = 1
        
        if seq in seq_user_dic:
            if user in seq_user_dic[seq]:
                seq_user_dic[seq][user] +=1
            else:
                seq_user_dic[seq][user] = 1
        else:
            seq_user_dic[seq] = {user: 1}

CPU times: user 2min 45s, sys: 2.18 s, total: 2min 47s
Wall time: 2min 47s


In [58]:
#Doesn't help pruning early by user
%%time
for us, ses in user_seq_dic.items():
    for seq, sup in ses.items():
        if sup < 3: #or len(seq) < 2:
            del ses[seq]

CPU times: user 1min 5s, sys: 144 ms, total: 1min 5s
Wall time: 1min 5s


In [ ]:
#Don't need
%%time
seq_user_dic = {}
for us, ses in user_seq_dic.items():
    for seq, sup in ses.items():
        if seq in seq_user_dic:
            seq_user_dic[seq][us] = sup
        else:
            seq_user_dic[seq] = {us: sup}
        del user_seq_dic[us][seq]

In [23]:
len(seq_user_dic)

26735005

In [ ]:
%%time
for seq, us in seq_user_dic.items():
    if len(us) > 1:
        del seq_user_dic[seq]

CPU times: user 10min 33s, sys: 23min, total: 33min 34s
Wall time: 5h 6min 37s


In [11]:
%%time
sups = []
for seq, us in seq_user_dic.items():
    for user, sup in us.items():
        sups.append(sup)

CPU times: user 1min 52s, sys: 412 ms, total: 1min 52s
Wall time: 1min 52s


In [12]:
Counter(sups)

Counter({1: 21288965,
         2: 951600,
         3: 274541,
         4: 124020,
         5: 67957,
         6: 42301,
         7: 28837,
         8: 20701,
         9: 15193,
         10: 11854,
         11: 9311,
         12: 7464,
         13: 6126,
         14: 5244,
         15: 4322,
         16: 3813,
         17: 3267,
         18: 2862,
         19: 2523,
         20: 2110,
         21: 1817,
         22: 1717,
         23: 1528,
         24: 1379,
         25: 1271,
         26: 1189,
         27: 1058,
         28: 968,
         29: 889,
         30: 823,
         31: 732,
         32: 741,
         33: 627,
         34: 601,
         35: 545,
         36: 485,
         37: 511,
         38: 468,
         39: 403,
         40: 385,
         41: 336,
         42: 309,
         43: 304,
         44: 263,
         45: 272,
         46: 256,
         47: 207,
         48: 192,
         49: 193,
         50: 188,
         51: 190,
         52: 151,
         53: 145,
         54:

In [62]:
%%time
for seq, us in seq_user_dic.items():
    if max(us.values()) < 5:
        del seq_user_dic[seq]

CPU times: user 496 ms, sys: 236 ms, total: 732 ms
Wall time: 689 ms


In [63]:
seq_user_dic

{(810, 815, 810, 690, 882): {2048: 1, 2122: 1, 3193: 6, 3324: 1},
 (690, 810, 690, 882): {787: 2,
  1113: 1,
  1373: 3,
  1914: 3,
  2078: 3,
  2122: 2,
  3193: 11,
  3302: 2,
  3324: 1},
 (16154, 2184, 16184, 16154): {3165: 15},
 (5, 5, 14): {44: 1,
  54: 1,
  231: 6,
  247: 1,
  380: 1,
  387: 1,
  390: 1,
  485: 1,
  537: 2,
  754: 1,
  967: 6,
  1053: 1,
  1056: 1,
  1173: 1,
  1377: 3,
  1575: 1,
  1657: 1,
  1665: 1,
  1743: 1,
  1750: 1,
  1845: 4,
  1866: 1,
  2082: 1,
  2116: 1,
  2122: 1,
  2296: 1,
  2377: 1,
  2603: 1,
  2631: 1,
  2676: 1,
  2861: 2,
  2960: 1,
  3108: 1,
  3305: 1,
  3333: 1},
 (3399, 3402): {121: 32, 538: 31},
 (16154, 8665, 16154, 8665, 8665): {3165: 5},
 (143, 4273, 143): {770: 4, 2238: 9, 2676: 1, 2783: 1},
 (11, 8383, 17307): {3324: 8},
 (735, 735, 726): {244: 2,
  380: 1,
  537: 8,
  689: 4,
  782: 4,
  1048: 2,
  1531: 9,
  1702: 2,
  1807: 6,
  1826: 10,
  1949: 2,
  1962: 1,
  2223: 1,
  2820: 6,
  3111: 1},
 (16183, 16183): {537: 3, 3165: 5},
 (

In [64]:
Counter([len(k) for k, v in seq_user_dic.items()])

Counter({1: 3471, 2: 19573, 3: 34836, 4: 31407, 5: 21833})

In [65]:
Counter([len(v) for k, v in seq_user_dic.items()])

Counter({1: 40216,
         2: 12853,
         3: 7006,
         4: 5299,
         5: 3913,
         6: 3059,
         7: 2540,
         8: 2239,
         9: 1937,
         10: 1651,
         11: 1452,
         12: 1393,
         13: 1312,
         14: 1358,
         15: 1282,
         16: 1237,
         17: 1207,
         18: 954,
         19: 922,
         20: 810,
         21: 794,
         22: 707,
         23: 747,
         24: 696,
         25: 643,
         26: 599,
         27: 639,
         28: 570,
         29: 522,
         30: 481,
         31: 441,
         32: 433,
         33: 416,
         34: 417,
         35: 323,
         36: 335,
         37: 291,
         38: 299,
         39: 259,
         40: 262,
         41: 250,
         42: 226,
         43: 254,
         44: 249,
         45: 253,
         46: 194,
         47: 176,
         48: 183,
         49: 188,
         50: 176,
         51: 185,
         52: 181,
         53: 147,
         54: 146,
         55: 157,


In [66]:
len(seq_user_dic)

111120

In [67]:
%%time
train_df_sites = train_data[['site' + str(c) for c in range(1,10+1)]].fillna(0).astype('int')
train_data["sequence"] = ""
for r in range(len(train_df_sites)):
    session = train_df_sites.iloc[r].as_matrix().astype(int)
    session = np.delete(session, np.where(session.astype(int) == 0)[0])

    ses_seq = []

    ses_seq_set = set()
    

    it = np.nditer(session, flags=['f_index'])
    while not it.finished:
        #ses_seq_set.add(tuple([int(it[0])]))
        asession = session[it.index+1:].copy()
        if len(asession) > 0:      
            ita = np.nditer(asession, flags=['f_index'])
            while not ita.finished:
                ses_seq_set.add(tuple([int(it[0]), int(ita[0])]))
                bsession = asession[ita.index+1:]
                if len(bsession) > 0:
                    itb = np.nditer(bsession, flags=['f_index'])
                    while not itb.finished:
                        ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0])]))
                        csession = bsession[itb.index+1:]
                        if len(csession) > 0:
                            itc = np.nditer(csession, flags=['f_index'])
                            while not itc.finished:
                                ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0])]))
                                dsession = csession[itc.index+1:]
                                if len(dsession) > 0:
                                    itd = np.nditer(dsession, flags=['f_index'])
                                    while not itd.finished:
                                        ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0])]))
                                        esession = dsession[itd.index+1:]
                                        if len(esession) > 0:
                                            ite = np.nditer(esession, flags=['f_index'])
                                            while not ite.finished:
                                                ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0]), int(ite[0])]))
                                                fsession = esession[ite.index+1:]
                                                if len(fsession) > 0:
                                                    itf = np.nditer(fsession, flags=['f_index'])
                                                    while not itf.finished:
                                                        ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0]), \
                                                                               int(ite[0]), int(itf[0])]))
                                                        itf.iternext()
                                                ite.iternext()
                                        itd.iternext()
                                itc.iternext()
                        itb.iternext()
                ita.iternext()
        it.iternext()
    
    for seq in ses_seq_set:
        if seq in seq_user_dic:
            #if int(train_data.iloc[r,-3]) != 0:
                #train_data.set_value(r, -3, int(seq_user_dic[seq].keys()[0]))
            ses_seq.append("_".join(np.array(seq).astype(str).tolist()))
                          
    if len(ses_seq):
        train_data.set_value(r, -1, " ".join(ses_seq), takeable=True)

CPU times: user 4min 15s, sys: 148 ms, total: 4min 15s
Wall time: 4min 15s


In [68]:
%%time
test_df_sites = test_data[['site' + str(c) for c in range(1,10+1)]].fillna(0).astype('int')
test_data["sequence"] = ""
for r in range(len(test_df_sites)):
    session = test_df_sites.iloc[r].as_matrix().astype(int)
    session = np.delete(session, np.where(session.astype(int) == 0)[0])

    ses_seq = []

    ses_seq_set = set()
    

    it = np.nditer(session, flags=['f_index'])
    while not it.finished:
        #ses_seq_set.add(tuple([int(it[0])]))
        asession = session[it.index+1:].copy()
        if len(asession) > 0:      
            ita = np.nditer(asession, flags=['f_index'])
            while not ita.finished:
                ses_seq_set.add(tuple([int(it[0]), int(ita[0])]))
                bsession = asession[ita.index+1:]
                if len(bsession) > 0:
                    itb = np.nditer(bsession, flags=['f_index'])
                    while not itb.finished:
                        ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0])]))
                        csession = bsession[itb.index+1:]
                        if len(csession) > 0:
                            itc = np.nditer(csession, flags=['f_index'])
                            while not itc.finished:
                                ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0])]))
                                dsession = csession[itc.index+1:]
                                if len(dsession) > 0:
                                    itd = np.nditer(dsession, flags=['f_index'])
                                    while not itd.finished:
                                        ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0])]))
                                        esession = dsession[itd.index+1:]
                                        if len(esession) > 0:
                                            ite = np.nditer(esession, flags=['f_index'])
                                            while not ite.finished:
                                                ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0]), int(ite[0])]))
                                                fsession = esession[ite.index+1:]
                                                if len(fsession) > 0:
                                                    itf = np.nditer(fsession, flags=['f_index'])
                                                    while not itf.finished:
                                                        ses_seq_set.add(tuple([int(it[0]), int(ita[0]), int(itb[0]), int(itc[0]), int(itd[0]), \
                                                                               int(ite[0]), int(itf[0])]))
                                                        itf.iternext()
                                                ite.iternext()
                                        itd.iternext()
                                itc.iternext()
                        itb.iternext()
                ita.iternext()
        it.iternext()
    
    for seq in ses_seq_set:
        if seq in seq_user_dic:
            ses_seq.append("_".join(np.array(seq).astype(str).tolist()))
                          
    if len(ses_seq):
        test_data.set_value(r, -1, " ".join(ses_seq), takeable=True)

CPU times: user 1min 32s, sys: 56 ms, total: 1min 32s
Wall time: 1min 32s


In [19]:
len(test_data)

41177

In [69]:
train_data = pd.DataFrame(train_data[train_data.sequence != ""])

In [70]:
train_data

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,sh2,sh3,sh4,sh5,sh6,sh7,sh8,sh9,sh10,sequence
session_id,,,,,,,,,,,,,,,,,,,,,
1,8,2014-01-04 08:44:50,11.0,2014-01-04 08:44:50,82.0,2014-01-04 08:45:19,68.0,2014-01-04 08:45:25,8.0,2014-01-04 08:45:25,...,11z8,82z8,68z8,8z8,8393z8,8403z8,932z8,3260z8,8z8,8403_932_8 82_8393_8403 8_8_8393 8_932_8 11_84...
2,111,2014-03-18 10:33:20,78.0,2014-03-18 10:33:31,151.0,2014-03-18 10:33:31,111.0,2014-03-18 10:33:31,1401.0,2014-03-18 10:33:31,...,78z10,151z10,111z10,1401z10,151z10,1375z10,38z10,1401z10,97z10,111_1401 151_38_97 111_78_111 78_111_38 111_78...
3,11,2014-12-02 13:13:41,3187.0,2014-12-02 13:13:41,132.0,2014-12-02 13:13:42,496.0,2014-12-02 13:13:42,1969.0,2014-12-02 13:13:45,...,3187z13,132z13,496z13,1969z13,504z13,3187z13,82z13,3191z13,3184z13,3191_3184 82_3184 3187_3187 496_82 132_3184 82...
4,668,2014-02-14 15:16:45,1965.0,2014-02-14 15:17:13,598.0,2014-02-14 15:20:47,1965.0,2014-02-14 15:21:13,284.0,2014-02-14 15:21:14,...,1965z15,598z15,1965z15,284z15,668z15,38z15,4451z15,4537z15,11z15,284_38 1965_1965_668 4537_11 284_4537 668_1965...
5,1943,2014-03-17 15:19:40,1943.0,2014-03-17 15:20:10,1943.0,2014-03-17 15:21:40,1943.0,2014-03-17 15:22:10,1943.0,2014-03-17 15:22:39,...,1943z15,1943z15,1943z15,1943z15,1952z15,1952z15,1943z15,1943z15,1943z15,1952_1952 1952_1952_1943_1943 1943_1952 1943_1...
6,2941,2013-02-12 12:09:43,2951.0,2013-02-12 12:09:44,2958.0,2013-02-12 12:09:46,2993.0,2013-02-12 12:09:54,162.0,2013-02-12 12:09:54,...,2951z12,2958z12,2993z12,162z12,2967z12,85z12,7z12,2967z12,2967z12,162_7 85_7 162_85 2951_2958 2941_2958 2941_2951
7,9996,2014-03-19 10:15:50,9996.0,2014-03-19 10:16:50,307.0,2014-03-19 10:17:19,9996.0,2014-03-19 10:17:20,280.0,2014-03-19 10:17:29,...,9996z10,307z10,9996z10,280z10,9996z10,9996z10,307z10,9996z10,9996z10,307_307_9996_9996 9996_9996_9996_307_9996 9996...
8,19823,2013-04-12 10:52:36,19823.0,2013-04-12 10:52:37,1510.0,2013-04-12 10:53:37,32.0,2013-04-12 10:54:49,19808.0,2013-04-12 10:55:01,...,19823z10,1510z10,32z10,19808z10,8z10,567z10,567z10,654z10,567z10,32_8 1510_32 1510_19808
10,32,2014-01-16 11:35:39,32.0,2014-01-16 11:35:49,561.0,2014-01-16 11:35:50,69.0,2014-01-16 11:36:17,32.0,2014-01-16 11:36:34,...,32z11,561z11,69z11,32z11,329z11,329z11,69z11,69z11,329z11,32_32_329 32_69_32 32_561_69 69_329_329 69_329...


In [27]:
%%time
# Train set sequence
train_data["sequence"] = ""
for r, row in train_data.iterrows():
    trie = make_trie(train_data[range(0,10)].iloc[r].as_matrix().astype(str))
    ses_seq = set()
    #for us, hses in user_seq_dic.items():
    for seq, us in seq_user_dic.items():
        #if row["target"] in us:
        sequence = np.array(seq).astype(str).tolist()
        if in_trie(trie, sequence):
            ses_seq |= set(["_".join(sequence)])
    if len(ses_seq):
        train_data.set_value(r, -1, " ".join(ses_seq), takeable=True)

CPU times: user 4h 33min 33s, sys: 56.7 s, total: 4h 34min 29s
Wall time: 4h 32min 57s


In [96]:
test_data["sequence"] = "-1"

In [28]:
%%time
#DON'T RUN TOO LONG
test_data["sequence"] = ""
for r, row in test_data.iterrows():
    trie = make_trie(test_data[range(0,10)].iloc[r].as_matrix().astype(str))
    ses_seq = set()
    for seq, us in seq_user_dic.items():
        sequence = np.array(seq).astype(str).tolist()
        if in_trie(trie, sequence):
            ses_seq |= set(["_".join(sequence)])
    if len(ses_seq):
        test_data.set_value(r, -1, " ".join(ses_seq), takeable=True)

CPU times: user 1h 42min 33s, sys: 21.2 s, total: 1h 42min 54s
Wall time: 1h 42min 20s


In [72]:
%%time
for r, row in train_data[train_data.sequence == ""].iterrows():
    session = train_data[range(0,10)].iloc[r].as_matrix().astype(int)
    session = np.delete(session, np.where(session.astype(int) == 0)[0])
    train_data.set_value(r, -1, " ".join(session.astype(str).tolist()), takeable=True)
    #train_data.set_value(r, -1, "-1", takeable=True)

CPU times: user 156 ms, sys: 564 ms, total: 720 ms
Wall time: 722 ms


In [73]:
%%time
for r, row in test_data[test_data.sequence == ""].iterrows():
    session = test_data[range(0,10)].iloc[r].as_matrix().astype(int)
    session = np.delete(session, np.where(session.astype(int) == 0)[0])
    #test_data.set_value(r, -1, " ".join(session.astype(str).tolist()), takeable=True)
    test_data.set_value(r, -1, "-1", takeable=True)

CPU times: user 28 ms, sys: 44 ms, total: 72 ms
Wall time: 331 ms


In [133]:
train_data.to_csv("kaggle_data/full_train_w8_seq_s_clean.csv", index=False)
test_data.to_csv("kaggle_data/full_test_w8_seq_s_clean.csv", index=False)

In [71]:
train_test_df = pd.concat([train_data, test_data])

session_length = 10

y = train_data["user_id"]
y_weights = [1.0] * 550

train_test_df_sites = train_test_df.sequence

tfidf = CountVectorizer(token_pattern='\w+').fit(train_test_df_sites.as_matrix())
X_train_test_seq_sparse = tfidf.transform(train_test_df_sites.as_matrix())

#train_test_df_sites = train_test_df[['site' + str(c) for c in range(1,10+1)]].fillna(0).astype('int')
#train_test_df_sites_array = [" ".join([str(s) for s in train_test_df_sites.as_matrix()[i].astype(str) if int(s) != 0]) \
 #                                                             for i in range(train_test_df_sites.shape[0])]


In [72]:
X_train_sites_seq_sparse = X_train_test_seq_sparse[:len(train_data)]
X_test_sites_seq_sparse = X_train_test_seq_sparse[len(train_data):]

class_encoder = LabelEncoder().fit(y.astype('str'))
y_for_vw = class_encoder.transform(y.astype('str')) + 1

sites_seq_columns_num = X_train_test_seq_sparse.shape[1]

In [73]:
%%time
#5
#train_test_df = pd.concat([train_data, test_data])

#session_length = 10
#train_index_dup = list(train_data[train_data.duplicated(subset=['site' + str(c) for c in range(1,session_length+1)], keep=False)]\
                       #[['site' + str(c) for c in range(1,10+1)]+["target"]].index)

#test_index_dup = list(test_data[test_data.duplicated(subset=['site' + str(c) for c in range(1,session_length+1)], keep=False)]\
                       #[['site' + str(c) for c in range(1,10+1)]].index)
#train_duplicates_mask = np.transpose([np.in1d(train_index_full, train_index_dup).astype(int)])
#test_duplicates_mask = np.transpose([np.in1d(test_index_full, test_index_dup).astype(int)])

#y = train_data["target"]

train_test_df_sites = train_test_df[['site' + str(c) for c in range(1,10+1)]].fillna(0).astype('int')
train_test_df_sites_array = [" ".join([str(s) for s in train_test_df_sites.as_matrix()[i].astype(str) if int(s) != 0]) \
                                                             for i in range(train_test_df_sites.shape[0])]

tfidf2 = TfidfVectorizer(token_pattern='\w+').fit(train_test_df_sites_array) #TfidfVectorizer()
X_train_test_sparse = tfidf2.transform(train_test_df_sites_array)

X_train_sites_sparse = X_train_test_sparse[:len(train_data)]
X_test_sites_sparse = X_train_test_sparse[len(train_data):]

#class_encoder = LabelEncoder().fit(y.astype('str'))
#y_for_vw = class_encoder.transform(y.astype('str')) + 1

sites_columns_num = X_train_test_sparse.shape[1]
#inv_vocabulary = {v: int(re.search("s_(\d+)$", k).group(1)) for k, v in tfidf.vocabulary_.iteritems()}

#y_weights = [(np.sum(Counter(y_for_vw).values()) - v + min((Counter(y_for_vw).values())))/ \
            #float(np.sum(Counter(y_for_vw).values())) for k, v in sorted(Counter(y_for_vw).items())]

#y_weights = [1.0] * 550

CPU times: user 4.72 s, sys: 24 ms, total: 4.75 s
Wall time: 4.73 s


In [400]:
%%time
row = []
col = []
data = []
for r, p in train_preds.items():
    if len(p) == 1:
        row.append(r)
        col.append(np.array(class_encoder.transform([str(p[0])])+1)[0])
        data.append(100)
    elif len(p) < 4:
        for us in p:
            row.append(r)
            col.append((np.array(class_encoder.transform([str(us)])+1)[0]))
            data.append(round(100./len(p), 0))
print max(col)
train_preds_sparse = csr_matrix((data, (row, col)), shape=(train_data.shape[0], 551), dtype=float)[:,1:]

row = []
col = []
data = []
for r, p in test_preds.items():
    if len(p) == 1:
        row.append(r)
        col.append(np.array(class_encoder.transform([str(p[0])])+1)[0])
        data.append(100)
    elif len(p) < 4:
        for us in p:
            row.append(r)
            col.append((np.array(class_encoder.transform([str(us)])+1)[0]))
            data.append(round(100./len(p), 0))
test_preds_sparse = csr_matrix((data, (row, col)), shape=(test_data.shape[0], 551), dtype=float)[:,1:]

550
CPU times: user 24 s, sys: 12 ms, total: 24 s
Wall time: 24 s


In [74]:
%%time
#6
#mycolumns = [label for label in test_data[range(20, 40)]]

#train_features, test_features = features_to_sparse(train_data, test_data, mycolumns)
train_features = None
test_features = None

X_train_sparse, X_test_sparse = combine_sites_features_sparse(X_train_sites_sparse, X_train_sites_seq_sparse, train_features,\
                                                             X_test_sites_sparse, X_test_sites_seq_sparse, test_features)#, test_preds_sparse, \
                                                              #train_site_times_sparse, test_site_times_sparse, \
                                                              #train_site_rels_sparse, test_site_rels_sparse)
                                                             #train_site_sequence, test_site_sequence)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y_for_vw, test_size=0.3, stratify=y_for_vw)

#y_train_weights = [(np.sum(Counter(y_train).values()) - v + min((Counter(y_train).values()))) / \
                   #float(np.sum(Counter(y_train).values())) for k, v in sorted(Counter(y_train).items())]

y_train_weights = [1.0] * 550

CPU times: user 1.6 s, sys: 24 ms, total: 1.63 s
Wall time: 1.64 s


In [75]:
%%time
#7
train_part_vw = sparse_matrix_to_vw(X_train, sites_columns_num, sites_seq_columns_num, y_train, weights=y_train_weights, mycolumns=mycolumns)
valid_vw = sparse_matrix_to_vw(X_valid, sites_columns_num, sites_seq_columns_num, y_valid, mycolumns=mycolumns)
train_vw = sparse_matrix_to_vw(X_train_sparse, sites_columns_num, sites_seq_columns_num, y_for_vw, weights=y_weights, mycolumns=mycolumns)
test_vw = sparse_matrix_to_vw(X_test_sparse, sites_columns_num, sites_seq_columns_num, mycolumns=mycolumns)

CPU times: user 8min 1s, sys: 776 ms, total: 8min 2s
Wall time: 8min 2s


In [344]:
X_train.shape[1] - sites_columns_num - 20

0

In [161]:
X_train_sparse[2]

<1x14372 sparse matrix of type '<type 'numpy.float64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [276]:
X_train

<74233x14381 sparse matrix of type '<type 'numpy.float64'>'
	with 2249519 stored elements in Compressed Sparse Row format>

In [120]:
X_valid

<31815x14372 sparse matrix of type '<type 'numpy.float64'>'
	with 944818 stored elements in Compressed Sparse Row format>

In [134]:
X_train[140, 14370]

0.84615384615384615

In [149]:
sites_columns_num

14352

In [162]:
X_train_sparse[2].nonzero()

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 array([14354, 14355, 14356, 14357, 14358, 14360, 14361, 14362, 14363,
        14364, 14365, 14366, 14369], dtype=int32))

In [277]:
np.where(np.in1d(range(74233), train_part_vw["sites"].keys()) == False)[0]

array([], dtype=int64)

In [35]:
for key, val in sorted(train_part_vw.items()):
    print key, len(val)

bot30_portion 0
day_of_week 0
daytime 0
fb_portion 0
lable_weights 66723
lables 66723
prediction 0
sequence 65132
session_timespan 0
site_longest_time 0
siterels 0
sites 66723
sitetimes 0
start_hour 0
top30_portion 0
unique_site 0
youtube_portion 0


In [136]:
y_train.shape

(74233,)

In [341]:
class_encoder.inverse_transform(314)

'280'

In [342]:
train_vw["lables"][28]

'315'

In [343]:
site_dic[14369]

{280}

In [344]:
test_vw["sites"][28]

' |site 107:0.321329333373 111:0.445374067933 1371:0.321329333373 1375:0.318494022952 151:0.471352792253 78:0.504314022264 8:0.131119518638'

In [53]:
valid_vw["prediction"]

{}

In [365]:
test_vw["siterels"][32771]

' |lsiterels 9:0.00322 11:0.00197 32:0.00188 65:0.00212 2083:0.01493 2119:0.01389 10125:0.125 13083:1.0'

In [366]:
test_vw["sitetimes"][32771]

' |isitetime 9:0.11817 11:0.12408 65:0.69645 2083:0.01551 10125:0.00074 13083:0.04505'

In [348]:
len(test_vw["prediction"])

7916

In [37]:
folder = 'kaggle_data/'
handler = '_idf_w10_orig_seq'

In [19]:
print "Saving vw files"
#with open(folder+'train_part'+handler+'.pkl', 'wb') as f:
    #pickle.dump(train_part_vw, f)
#with open(folder+'valid'+handler+'.pkl', 'wb') as f:
    #pickle.dump(valid_vw, f)
with open(folder+'train'+handler+'.pkl', 'wb') as f:
    pickle.dump(train_vw, f)
#with open(folder+'test'+handler+'.pkl', 'wb') as f:
    #pickle.dump(test_vw, f)
#with open(folder+'class_encoder'+handler+'.pkl', 'wb') as f:
    #pickle.dump(class_encoder, f)

#y.to_csv(folder+'y'+handler+'.csv', index=False, header=False)
#pd.DataFrame(y_train).to_csv(folder+'y_train'+handler+'.csv', index=False, header=False)
#pd.DataFrame(y_valid).to_csv(folder+'y_valid'+handler+'.csv', index=False, header=False)

Saving vw files


KeyboardInterrupt: 

In [76]:
%%time
#8

keys = ['sequence']

vw_to_file(train_part_vw["sites"], folder+'train_part'+handler+'.vw', \
           features={x:train_part_vw[x] for x in keys}, \
           lables=train_part_vw["lables"], lable_weights=train_part_vw["lable_weights"], quiet=True)
vw_to_file(valid_vw["sites"], folder+'valid'+handler+'.vw', features={x:valid_vw[x] for x in keys}, \
           lables=valid_vw["lables"], quiet=True)
vw_to_file(train_vw["sites"], folder+'train'+handler+'.vw', features={x:train_vw[x] for x in keys}, \
           lables=train_vw["lables"], lable_weights=train_vw["lable_weights"], quiet=True)
vw_to_file(test_vw["sites"], folder+'test'+handler+'.vw', features={x:test_vw[x] for x in keys}, quiet=True)

CPU times: user 548 ms, sys: 112 ms, total: 660 ms
Wall time: 696 ms


In [96]:
f = open(folder+'train_part'+handler+'.vw')
train_part_file = f.readlines()
f.close()

f = open(folder+'train'+handler+'.vw')
train_file = f.readlines()
f.close()

f = open(folder+'valid'+handler+'.vw')
valid_file = f.readlines()
f.close()

f = open(folder+'test'+handler+'.vw')
test_file = f.readlines()
f.close()

In [75]:
len(valid_file)

32749

In [121]:
%%time
model = VW(oaa=550, passes=20, b=28, convert_to_vw=False, \
          cubic="sbc", q="sd", quiet=False, l=0.45, decay_learning_rate=0.9, loss_function="squared")
model.fit(train_part_file)
vw_pred = model.predict(valid_file)
print accuracy_score(y_valid, vw_pred)

0.540471311475
CPU times: user 43 s, sys: 9.55 s, total: 52.6 s
Wall time: 48.7 s


In [153]:
vw_pred = pd.read_csv("kaggle_data/28vw_submission_idf_w8_1.csv")

In [ ]:
validation = pd.DataFrame(np.transpose([y_valid]), columns=["true"])

In [58]:
vw_pred = pd.DataFrame(np.transpose([vw_pred]).astype(int), columns=["vw_pred"])

In [126]:
class_encoder.inverse_transform(516)

'783'

In [154]:
#validation = pd.concat([validation, pd.DataFrame(np.transpose([vw_pred]).astype(int), columns=["vw_pred"])], axis=1)
ind = []
val = []
for i in range(len(test_file)):
    if i in test_vw["prediction"]:
        m = re.search("(\d+):100", test_vw["prediction"][i])
        if m: 
            ind.append(i)
            val.append(class_encoder.inverse_transform(int(m.group(1))-1))
            #val.append(int(m.group(1)))
        else:
            ind.append(i)
            val.append(0)
    else:
        ind.append(i)
        val.append(0)
preds = pd.DataFrame(np.transpose([val]), columns=["mypred"])
validation = pd.concat([vw_pred, preds], axis=1)


In [156]:
def result(row):
    if int(row.mypred) == 0:
        return row["user_id"]
    else:
        return str(row["mypred"])

In [157]:
validation

,session_id,user_id,mypred
0,1,537,0
1,2,783,783
2,3,1645,0
3,4,1962,0
4,5,631,631
5,6,1647,0
6,7,537,0
7,8,1113,0
8,9,3342,0
9,10,1505,0


In [155]:
validation[(validation.mypred.astype(int) != 0) & (validation.user_id.astype(int) != validation.mypred.astype(int))]

,session_id,user_id,mypred
31,32,494,3075
328,329,558,3324
356,357,2336,231
396,397,537,783
465,466,2902,3165
869,870,2738,2639
971,972,2238,2693
1154,1155,467,2942
1162,1163,2946,3065
1185,1186,2971,860


In [158]:
validation["result"] = validation.apply(lambda row: result(row), axis=1)

In [159]:
validation.drop(["user_id", "mypred"], axis=1).to_csv("kaggle_data/43_vw_submission_manual_pred.csv", index=False, header=["session_id", "user_id"])

In [83]:
vw_test_pred = pd.DataFrame(validation["result"])
t_submission = pd.DataFrame(vw_test_pred.astype(int)-1)
vw_subm = class_encoder.inverse_transform(t_submission)

In [85]:
write_to_submission_file(vw_subm,
             'kaggle_data/40vw_submission_pred.csv')
print "Finished creating submission.\n"

Finished creating submission.



In [77]:
%%time
#9
!vw --oaa=550 -d {folder}train_part{handler}.vw \
-f {folder}initial_model{handler}.model -b 27 -c -k \
--passes=5 -l 0.45 --decay_learning_rate=0.9 \
--keep "z"

!vw -i {folder}initial_model{handler}.model  -t -d {folder}valid{handler}.vw \
-p {folder}vw_valid_pred{handler}.csv --quiet

vw_valid_pred = pd.read_csv(folder+'vw_valid_pred'+handler+'.csv', header=None)
accuracy = accuracy_score(y_valid, vw_valid_pred.values)
print "Accuracy:", accuracy

using namespaces beginning with: z 
final_regressor = kaggle_data/initial_model_idf_w10_orig_seq.model
Num weight bits = 27
learning rate = 0.45
initial_t = 0
power_t = 0.5
decay_learning_rate = 0.9
creating cache_file = kaggle_data/train_part_idf_w10_orig_seq.vw.cache
Reading datafile = kaggle_data/train_part_idf_w10_orig_seq.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      545        1       51
1.000000 1.000000            2            2.0       64      545       25
1.000000 1.000000            4            4.0       88      545      127
1.000000 1.000000            8            8.0      268      417       13
1.000000 1.000000           16           16.0      393      372        7
1.000000 1.000000           32           32.0      124       50       60
0.968750 0.937500           64           64.0      440      266     

In [282]:
%%time
!vw --oaa=550 -d {folder}train_part{handler}.vw \
-f {folder}initial_model{handler}.model -b 28 -c -k \
--passes=30 --decay_learning_rate 0.9 --initial_t 0.002337045080352835 \
-l 0.5416950450219994 \
--power_t 0.5 --loss_function='logistic' --l1 1e-11 --l2 1e-11 \
--cubic="sbc"  \
--keep "s" --keep "b" --keep "c" --keep "l" --keep "a" \
--stage_poly --batch_sz {len(train_part_file)/6} --batch_sz_no_doubling

creating cubic features for triples: sbc 
using namespaces beginning with: s b c l a 
using l1 regularization = 1e-11
using l2 regularization = 1e-11
final_regressor = kaggle_data/initial_model_idf_w8_seq.model
Num weight bits = 28
learning rate = 0.541695
initial_t = 0.00233705
power_t = 0.5
decay_learning_rate = 0.9
creating cache_file = kaggle_data/train_part_idf_w8_seq.vw.cache
Reading datafile = kaggle_data/train_part_idf_w8_seq.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0       69        1       55
1.000000 1.000000            2            2.0      276       69       11
1.000000 1.000000            4            4.0      364      276       11
1.000000 1.000000            8            8.0      145       12       51
1.000000 1.000000           16           16.0      291      276       45
1.000000 1.000000           32  

In [408]:
!vw -i {folder}initial_model{handler}.model  -t -d {folder}valid{handler}.vw \
-p {folder}vw_valid_pred{handler}.csv --quiet

vw_valid_pred = pd.read_csv(folder+'vw_valid_pred'+handler+'.csv', header=None)
accuracy = accuracy_score(y_valid, vw_valid_pred.values)
print "Accuracy:", accuracy

Accuracy: 0.596499252889


## Confusion Matrix

In [404]:
countery = Counter(y_train)
confusion = {}

In [405]:
M = confusion_matrix(y_valid, vw_valid_pred)
M_normalized = M.astype('float') / M.sum(axis=1)[:, np.newaxis]
for (t,f), value in np.ndenumerate(M):
    if t != f and value > 0:
        confusion[tuple([t, f])] = value

In [371]:
one_confusion = {}
for k, v in confusion.items():
    if tuple([k[1], k[0]]) not in confusion:
        one_confusion[k] = v
two_confusion = {}
for k, v in confusion.items():
    if tuple([k[1], k[0]]) in confusion and tuple([k[1], k[0]]) not in two_confusion:
        two_confusion[k] = v

In [376]:
train_data

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,site_longest_time,start_hour,day_of_week,daytime,fb_portion,youtube_portion,top30_portion,bot30_portion,prediction,target
0,3537,3537,3537,5759,37,16,3537,47,17,53,...,5759,14,4,1,0.000000,0.0,0.000000,0.500000,0,280
1,47,37,53,56,3537,47,53,47,17,53,...,56,14,4,1,0.000000,0.0,0.000000,0.000000,0,280
2,22,53,17,6,22,22,22,22,90,22,...,17,14,4,1,0.000000,0.0,0.029412,0.000000,0,280
3,22,87,22,22,14326,14326,87,14326,14334,14326,...,22,14,4,1,0.000000,0.0,0.428571,0.571429,0,280
4,14334,14326,0,0,0,0,0,0,0,0,...,14334,14,4,1,0.000000,0.0,0.000000,1.000000,0,280
5,540,56,16,47,47,17,53,1027,347,16,...,53,15,4,1,0.000000,0.0,0.000000,0.000000,0,280
6,347,347,90,22,680,690,662,690,662,662,...,347,16,4,1,0.000000,0.0,0.115385,0.000000,0,280
7,690,38,680,690,662,680,690,662,662,690,...,38,16,4,1,0.000000,0.0,0.615385,0.000000,0,280
8,680,680,662,690,662,662,882,690,680,690,...,662,16,4,1,0.000000,0.0,0.000000,0.000000,0,280
9,2415,680,690,2415,662,882,2415,690,690,882,...,882,16,4,1,0.000000,0.0,0.000000,0.000000,0,280


In [378]:
for a, b in [pair for pair, val in sorted(two_confusion.items(), key=lambda t:t[1], reverse=True)]:
    train_data = train_data[train_data.target != a].copy()
    train_data = train_data[train_data.target != b].copy()

226 317
351 235
316 137
329 248
363 178
510 251
266 140
34 371
112 405
245 539
306 195
138 380
548 154
82 240
502 314
72 496
376 134
116 228
461 17
4 342
242 399
359 17
238 441
49 468
416 385
445 375
291 94
97 17
353 378
272 321
123 298
416 468
296 342
37 378
3 56
296 4
144 486
541 493
180 168
289 346
461 97
385 468
533 499
9 269
403 422
533 123
124 147
175 307
472 317
71 129
469 40
298 533
517 17
353 37
476 293
547 334
49 416
184 63
415 63
307 293
261 191
424 108
135 299
472 226
419 270
422 298
323 171
385 49
19 214
270 213
159 394
159 181
285 518
270 470
47 401
311 512
87 327
419 470
517 457
297 58
293 175
544 268
185 22
293 171
250 419
282 517
403 17
215 522
417 501
219 534
400 389
538 352
499 123
268 226
472 268
501 462
62 117
385 363
220 419
415 519
54 198
353 371
268 317
177 436
178 214
11 213
293 133
175 17
421 152
148 230
286 471
132 169
207 514
26 499
336 270
443 7
326 385
426 378
406 167
426 457
106 303
179 26
424 403
446 322
326 195
88 443
34 353
324 172
214 91
34 457
459 23

In [383]:
len(Counter(train_data.target))

477

In [374]:
np.mean(two_confusion.values())

4.1379310344827589

In [375]:
[[[tf[0], countery[tf[0]+1]], [tf[1], countery[tf[1]+1]], [val]] for tf, val in sorted(two_confusion.items(), \
                key=lambda t: t[1], reverse = True)]

[[[226, 944], [317, 942], [193]],
 [[351, 398], [235, 395], [124]],
 [[316, 505], [137, 502], [109]],
 [[329, 431], [248, 428], [103]],
 [[363, 412], [178, 414], [103]],
 [[510, 329], [251, 322], [87]],
 [[266, 316], [140, 317], [78]],
 [[34, 351], [371, 353], [69]],
 [[112, 232], [405, 232], [52]],
 [[245, 272], [539, 269], [49]],
 [[306, 147], [195, 146], [44]],
 [[138, 187], [380, 187], [43]],
 [[548, 170], [154, 171], [42]],
 [[82, 150], [240, 150], [40]],
 [[502, 170], [314, 170], [37]],
 [[72, 115], [496, 115], [33]],
 [[376, 137], [134, 135], [28]],
 [[116, 146], [228, 146], [26]],
 [[461, 186], [17, 377], [24]],
 [[4, 135], [342, 137], [23]],
 [[242, 87], [399, 87], [22]],
 [[359, 113], [17, 377], [22]],
 [[238, 106], [441, 104], [22]],
 [[49, 725], [468, 1586], [21]],
 [[416, 466], [385, 2691], [20]],
 [[445, 81], [375, 80], [20]],
 [[291, 69], [94, 68], [19]],
 [[97, 182], [17, 377], [19]],
 [[353, 235], [378, 153], [18]],
 [[272, 94], [321, 94], [17]],
 [[123, 199], [298, 26

In [406]:
pairs = [[class_encoder.inverse_transform(tf[0]), class_encoder.inverse_transform(tf[1])] \
         for tf, val in sorted(confusion.items(), key=lambda t: t[1], reverse = True)]

In [407]:
pairs

[['2824', '2336'],
 ['2336', '2824'],
 ['2820', '1807'],
 ['1807', '2820'],
 ['2366', '2971'],
 ['2434', '2874'],
 ['3060', '2004'],
 ['2971', '2366'],
 ['244', '762'],
 ['2874', '2434'],
 ['2004', '3060'],
 ['1812', '2524'],
 ['762', '244'],
 ['1180', '3102'],
 ['2524', '1812'],
 ['1665', '3260'],
 ['3102', '1180'],
 ['2414', '940'],
 ['3260', '1665'],
 ['940', '2414'],
 ['1808', '313'],
 ['974', '1877'],
 ['313', '1808'],
 ['2342', '17'],
 ['239', '1505'],
 ['728', '280'],
 ['1505', '239'],
 ['280', '728'],
 ['2746', '2161'],
 ['1877', '974'],
 ['17', '2342'],
 ['121', '538'],
 ['1413', '692'],
 ['178', '3119'],
 ['2991', '1200'],
 ['3119', '178'],
 ['692', '1413'],
 ['2383', '387'],
 ['1985', '2773'],
 ['2824', '557'],
 ['2642', '2954'],
 ['2681', '1737'],
 ['572', '2773'],
 ['2161', '2746'],
 ['538', '121'],
 ['387', '2383'],
 ['3165', '3328'],
 ['2336', '557'],
 ['197', '2773'],
 ['2943', '267'],
 ['3165', '537'],
 ['3219', '2394'],
 ['537', '3165'],
 ['2842', '2561'],
 ['1273', '

In [363]:
class_encoder.inverse_transform(226), class_encoder.inverse_transform(317)

('2336', '2824')

In [213]:
class_encoder.transform([str(673)]) + 1

array([494])

In [356]:
train_data[train_data.target == 2336]

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,site_longest_time,start_hour,day_of_week,daytime,fb_portion,youtube_portion,top30_portion,bot30_portion,prediction,target
33258,22,22,90,69,22,87,77,77,22,22,...,87,13,0,1,0.0,0.0,0.333333,0.000000,0,2336
33259,22,22,87,22,22,22,22,22,1447,22,...,87,13,0,1,0.0,0.0,0.054795,0.000000,0,2336
33260,64,1447,70,22,64,22,64,70,71,22,...,22,13,0,1,0.0,0.0,0.666667,0.000000,0,2336
33261,71,22,63,22,64,63,70,70,63,64,...,63,13,0,1,0.0,0.0,0.333333,0.000000,0,2336
33262,71,70,63,64,70,70,64,71,71,63,...,64,13,0,1,0.0,0.0,0.000000,0.000000,0,2336
33263,71,64,63,63,22,64,64,22,63,70,...,64,13,0,1,0.0,0.0,0.250000,0.000000,0,2336
33264,22,70,63,70,71,64,63,64,63,70,...,70,13,0,1,0.0,0.0,0.500000,0.000000,0,2336
33265,64,70,64,22,70,63,64,64,63,70,...,64,13,0,1,0.0,0.0,0.111111,0.000000,0,2336
33266,63,70,64,71,70,71,22,71,63,71,...,71,13,0,1,0.0,0.0,0.375000,0.000000,0,2336
33267,63,70,71,64,22,63,70,63,70,64,...,63,13,0,1,0.0,0.0,0.333333,0.000000,0,2336


In [355]:
train_data[train_data.target == 2824]

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,site_longest_time,start_hour,day_of_week,daytime,fb_portion,youtube_portion,top30_portion,bot30_portion,prediction,target
99472,22,22,22,90,69,87,77,22,22,87,...,87,13,0,1,0.0,0.0,0.360000,0.000000,0,2824
99473,22,87,22,69,77,77,22,22,22,69,...,69,13,0,1,0.0,0.0,1.000000,0.000000,0,2824
99474,22,69,22,64,1447,70,22,71,64,70,...,22,13,0,1,0.0,0.0,1.000000,0.000000,0,2824
99475,71,22,63,64,71,63,70,64,71,63,...,64,13,0,1,0.0,0.0,0.000000,0.000000,0,2824
99476,71,63,71,22,64,63,70,63,70,64,...,63,13,0,1,0.0,0.0,0.500000,0.000000,0,2824
99477,63,70,22,71,63,64,69,70,70,63,...,70,13,0,1,0.0,0.0,0.333333,0.000000,0,2824
99478,70,63,70,64,63,63,22,64,71,71,...,71,13,0,1,0.0,0.0,0.333333,0.000000,0,2824
99479,64,64,22,71,63,70,63,63,70,63,...,64,13,0,1,0.0,0.0,0.166667,0.000000,0,2824
99480,70,64,71,63,63,64,22,71,63,70,...,64,13,0,1,0.0,0.0,0.222222,0.000000,0,2824
99481,63,71,71,64,63,70,64,71,63,70,...,71,13,0,1,0.0,0.0,0.000000,0.000000,0,2824


In [43]:
df= pd.read_csv('kaggle_data/train_sessions.csv')


In [45]:
df[df.user_id == 2631]

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
5491,5492,509,2014-03-29 11:06:24,858.0,2014-03-29 11:06:24,11.0,2014-03-29 11:06:24,65.0,2014-03-29 11:06:24,268.0,...,2014-03-29 11:06:25,82.0,2014-03-29 11:06:25,241.0,2014-03-29 11:06:31,265.0,2014-03-29 11:06:32,241.0,2014-03-29 11:06:32,2631
6571,6572,32,2014-08-02 10:04:39,32.0,2014-08-02 10:04:40,8.0,2014-08-02 10:04:41,8.0,2014-08-02 10:04:53,32.0,...,2014-08-02 10:04:55,3087.0,2014-08-02 10:14:37,521.0,2014-08-02 10:33:42,523.0,2014-08-02 10:33:42,32.0,2014-08-02 10:33:42,2631
8631,8632,65,2014-01-25 10:30:10,260.0,2014-01-25 10:36:59,412.0,2014-01-25 10:36:59,1361.0,2014-01-25 10:41:10,77.0,...,2014-01-25 10:41:10,1321.0,2014-01-25 10:41:11,1378.0,2014-01-25 10:41:11,1104.0,2014-01-25 10:41:12,2891.0,2014-01-25 10:41:12,2631
12124,12125,523,2014-08-02 10:33:43,524.0,2014-08-02 10:33:43,526.0,2014-08-02 10:33:44,32.0,2014-08-02 10:34:05,13832.0,...,2014-08-02 10:35:01,13836.0,2014-08-02 10:35:52,32.0,2014-08-02 10:35:52,13836.0,2014-08-02 10:35:53,342.0,2014-08-02 10:35:53,2631
14795,14796,32,2014-08-02 11:27:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2631
15614,15615,32,2014-08-02 11:10:39,32.0,2014-08-02 11:10:42,13834.0,2014-08-02 11:10:43,11583.0,2014-08-02 11:10:44,13834.0,...,2014-08-02 11:10:45,11.0,2014-08-02 11:10:45,625.0,2014-08-02 11:10:45,184.0,2014-08-02 11:10:46,625.0,2014-08-02 11:10:46,2631
16642,16643,2889,2014-02-22 11:26:25,67.0,2014-02-22 11:27:57,7.0,2014-02-22 11:27:58,2889.0,2014-02-22 11:27:58,38.0,...,2014-02-22 11:27:58,88.0,2014-02-22 11:30:31,38.0,2014-02-22 11:30:31,7.0,2014-02-22 11:30:31,2889.0,2014-02-22 11:30:31,2631
16952,16953,544,2014-05-13 08:05:30,525.0,2014-05-13 08:05:52,15.0,2014-05-13 08:05:57,479.0,2014-05-13 08:10:54,5.0,...,2014-05-13 08:12:55,525.0,2014-05-13 08:14:13,544.0,2014-05-13 08:14:13,525.0,2014-05-13 08:14:22,14.0,2014-05-13 08:14:54,2631
20707,20708,8,2014-01-25 10:55:50,32.0,2014-01-25 10:57:02,32.0,2014-01-25 11:01:14,8.0,2014-01-25 11:01:15,32.0,...,2014-01-25 11:01:33,65.0,2014-01-25 11:01:34,13828.0,2014-01-25 11:01:34,38.0,2014-01-25 11:01:35,85.0,2014-01-25 11:01:35,2631
21597,21598,13840,2014-01-25 11:01:35,13828.0,2014-01-25 11:01:35,3430.0,2014-01-25 11:01:35,13833.0,2014-01-25 11:01:35,11.0,...,2014-01-25 11:01:36,13828.0,2014-01-25 11:01:36,8.0,2014-01-25 11:01:37,38.0,2014-01-25 11:01:37,55.0,2014-01-25 11:01:37,2631


In [46]:
df[df.user_id == 786]

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
2502,2503,1955,2014-08-02 11:06:25,38.0,2014-08-02 11:06:26,38.0,2014-08-02 11:06:27,55.0,2014-08-02 11:06:27,1955.0,...,2014-08-02 11:06:29,1955.0,2014-08-02 11:06:31,1955.0,2014-08-02 11:06:37,32.0,2014-08-02 11:07:45,1946.0,2014-08-02 11:07:46,786
3384,3385,184,2014-01-17 14:31:24,53.0,2014-01-17 14:31:27,47.0,2014-01-17 14:31:27,17.0,2014-01-17 14:31:27,8.0,...,2014-01-17 14:32:43,2141.0,2014-01-17 14:36:02,32.0,2014-01-17 14:38:34,32.0,2014-01-17 14:38:48,NaN,NaN,786
4684,4685,106,2014-03-29 10:03:15,106.0,2014-03-29 10:03:30,106.0,2014-03-29 10:03:31,32.0,2014-03-29 10:03:31,106.0,...,2014-03-29 10:03:34,13839.0,2014-03-29 10:07:08,13839.0,2014-03-29 10:07:10,3087.0,2014-03-29 10:23:30,NaN,NaN,786
5462,5463,32,2014-05-13 15:04:22,32.0,2014-05-13 15:04:23,32.0,2014-05-13 15:04:26,106.0,2014-05-13 15:04:31,32.0,...,2014-05-13 15:05:00,322.0,2014-05-13 15:05:01,322.0,2014-05-13 15:08:05,305.0,2014-05-13 15:08:08,322.0,2014-05-13 15:08:37,786
5553,5554,65,2014-01-25 11:01:34,13840.0,2014-01-25 11:01:35,13828.0,2014-01-25 11:01:35,3430.0,2014-01-25 11:01:35,88.0,...,2014-01-25 11:01:36,8.0,2014-01-25 11:01:36,55.0,2014-01-25 11:01:37,38.0,2014-01-25 11:01:37,13828.0,2014-01-25 11:01:38,786
7182,7183,32,2014-08-02 10:04:39,32.0,2014-08-02 10:04:40,8.0,2014-08-02 10:04:41,32.0,2014-08-02 10:04:54,32.0,...,2014-08-02 10:14:37,32.0,2014-08-02 10:33:29,32.0,2014-08-02 10:33:42,523.0,2014-08-02 10:33:42,521.0,2014-08-02 10:33:42,786
9003,9004,82,2014-03-29 11:06:25,11.0,2014-03-29 11:06:25,268.0,2014-03-29 11:06:30,268.0,2014-03-29 11:06:31,265.0,...,2014-03-29 11:06:31,265.0,2014-03-29 11:06:32,49.0,2014-03-29 11:06:32,8.0,2014-03-29 11:06:32,11.0,2014-03-29 11:06:33,786
10851,10852,32,2014-08-02 11:10:42,11583.0,2014-08-02 11:10:44,13834.0,2014-08-02 11:10:45,11.0,2014-08-02 11:10:45,625.0,...,2014-08-02 11:10:45,184.0,2014-08-02 11:10:46,11.0,2014-08-02 11:10:46,13834.0,2014-08-02 11:10:46,625.0,2014-08-02 11:10:46,786
13176,13177,322,2014-05-13 15:09:17,322.0,2014-05-13 15:29:13,322.0,2014-05-13 15:29:19,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,786
14755,14756,32,2014-08-02 11:07:46,1980.0,2014-08-02 11:07:47,111.0,2014-08-02 11:07:47,7.0,2014-08-02 11:07:48,55.0,...,2014-08-02 11:07:48,38.0,2014-08-02 11:07:48,88.0,2014-08-02 11:07:48,111.0,2014-08-02 11:07:48,78.0,2014-08-02 11:07:49,786


In [76]:
test_data[test_data.duplicated(subset=['time' + str(c) for c in range(1,session_length+1)], keep=False)]\
                           [['site' + str(c) for c in range(1,10+1)]+['time' + str(c) for c in range(1,10+1)]]

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
868,38,0,0,0,0,0,0,0,0,0,2013-11-28 13:36:53,0,0,0,0,0,0,0,0,0
2694,27,0,0,0,0,0,0,0,0,0,2014-02-17 17:01:28,0,0,0,0,0,0,0,0,0
3840,869,0,0,0,0,0,0,0,0,0,2014-03-18 08:18:39,0,0,0,0,0,0,0,0,0
6829,307,0,0,0,0,0,0,0,0,0,2014-08-04 16:15:38,0,0,0,0,0,0,0,0,0
7492,27,9,0,0,0,0,0,0,0,0,2013-11-21 14:08:05,2013-11-21 14:08:05,0,0,0,0,0,0,0,0
7985,869,0,0,0,0,0,0,0,0,0,2014-03-16 18:48:02,0,0,0,0,0,0,0,0,0
8388,58,25,0,0,0,0,0,0,0,0,2013-12-18 10:18:28,2013-12-18 10:18:28,0,0,0,0,0,0,0,0
9065,1945,0,0,0,0,0,0,0,0,0,2014-02-21 15:17:45,0,0,0,0,0,0,0,0,0
10144,9,0,0,0,0,0,0,0,0,0,2014-03-25 08:34:39,0,0,0,0,0,0,0,0,0
10833,27,9,0,0,0,0,0,0,0,0,2013-11-21 14:08:05,2013-11-21 14:08:05,0,0,0,0,0,0,0,0


In [287]:
feature_names = ['site' + str(i) for i in range(1,11)] + ['time' + str(i) for i in range(1,11)] + \
                ['time_diff' + str(j) for j in range(1,10)] + \
                ['session_timespan', '#unique_sites', 'site_longest_time', 'start_hour', 'day_of_week', 'daytime', 'fb_portion',\
                 'youtube_portion', 'top30_portion', 'bot30_portion', 'prediction', 'target']

In [288]:
# Find duplicates
def delete_dups_max(csv_files_mask, feature_names, site_freq_path="", site_index_path="", dataframe_csv="",
                                    session_length=10, window_size=10, session_time = 30, num_users_for_prediction = 1, sort_in_session=False):
    files = glob(csv_files_mask)
    
    used = {}
    
    for pair in pairs:
        pair_data = pd.DataFrame()
        counter = 1
        
        if pair[0] not in used or pair[1] not in used:
            files = ['kaggle_data/train/user'+pair[0]+'.csv', 'kaggle_data/train/user'+pair[1]+'.csv']
            counter = 1

            for userfile in files:
                user_id = re.search('user(\d+)\.csv', userfile).group(1)
                data = pd.read_csv(userfile, \
                                   parse_dates=[0], infer_datetime_format=True, names=["timestamp"+str(counter), "site"+str(counter)], header=0)
                data.sort_values("timestamp"+str(counter), inplace=True)
                data.reset_index(drop=True, inplace=True)
                pair_data = pd.concat([pair_data, data], axis=1)
                counter +=1
            
            pair_data = pair_data[(pair_data.timestamp1 != pair_data.timestamp2) & \
                                  (pair_data.site1 != pair_data.site2)]

            data1 = pair_data[["timestamp1", "site1"]].dropna().copy()
            data2 = pair_data[["timestamp2", "site2"]].dropna().copy()

            if pair[0] not in used:
                data1.to_csv('kaggle_data/new_train/user'+pair[0]+'.csv', index=False, header=["timestamp", "site"])
                used[pair[0]] = 0
            if pair[1] not in used:
                data2.to_csv('kaggle_data/new_train/user'+pair[1]+'.csv', index=False, header=["timestamp", "site"])
                used[pair[1]] = 0
    return None

In [289]:
delete_dups_max('kaggle_data/train/*',
                                   feature_names=feature_names, 
                                            site_freq_path="kaggle_data/site_freq.pkl", session_length=10, sort_in_session=True)

In [218]:
# Find duplicates
def delete_dups_medium(csv_files_mask, feature_names, site_freq_path="", site_index_path="", dataframe_csv="",
                                    session_length=10, window_size=10, session_time = 30, num_users_for_prediction = 1, sort_in_session=False):
    files = glob(csv_files_mask)
    
    used = {}
    
    for pair in pairs:
        pair_data = pd.DataFrame()
        counter = 1
        
        if pair[0] not in used or pair[1] not in used:
            files = ['kaggle_data/train/user'+pair[0]+'.csv', 'kaggle_data/train/user'+pair[1]+'.csv']
            counter = 1

            for userfile in files:
                user_id = re.search('user(\d+)\.csv', userfile).group(1)
                data = pd.read_csv(userfile, \
                                   parse_dates=[0], infer_datetime_format=True, names=["timestamp"+str(counter), "site"+str(counter)], header=0)
                data.sort_values("timestamp"+str(counter), inplace=True)
                data.reset_index(drop=True, inplace=True)

                data["weekday"+str(counter)] = data["timestamp"+str(counter)].apply(lambda x: int(x.isoweekday()))
                data["hour"+str(counter)] = data["timestamp"+str(counter)].apply(lambda x: int(x.hour))
                pair_data = pd.concat([pair_data, data], axis=1)
                counter +=1
            
            pair_data = pair_data[(pair_data.weekday1 != pair_data.weekday2) & \
                                  (pair_data.site1 != pair_data.site2) & (pair_data.hour1 != pair_data.hour2)]

            data1 = pair_data[["timestamp1", "site1"]].dropna().copy()
            data2 = pair_data[["timestamp2", "site2"]].dropna().copy()

            if pair[0] not in used:
                data1.to_csv('kaggle_data/new_train/user'+pair[0]+'.csv', index=False, header=["timestamp", "site"])
                used[pair[0]] = 0
            if pair[1] not in used:
                data2.to_csv('kaggle_data/new_train/user'+pair[1]+'.csv', index=False, header=["timestamp", "site"])
                used[pair[1]] = 0
    return None

In [217]:
delete_dups_medium('kaggle_data/train/*',
                                   feature_names=feature_names, 
                                            site_freq_path="kaggle_data/site_freq.pkl", session_length=10, sort_in_session=True)

In [416]:
# Find duplicates
def delete_dups_minimal2(csv_files_mask, feature_names, site_freq_path="", site_index_path="", dataframe_csv="",
                                    session_length=10, window_size=10, session_time = 30, num_users_for_prediction = 1, sort_in_session=False):
    files = glob(csv_files_mask)
    pair_data = pd.DataFrame()
    
    counter = 1
    
    used = {}
    
    for pair in pairs:
        
        if pair[0] not in used or pair[1] not in used:
            files = ['kaggle_data/train/user'+pair[0]+'.csv', 'kaggle_data/train/user'+pair[1]+'.csv']
            counter = 1

            for userfile in files:
                user_id = re.search('user(\d+)\.csv', userfile).group(1)
                data = pd.read_csv(userfile, \
                                   parse_dates=[0], infer_datetime_format=True)
                data.sort_values("timestamp", inplace=True)
                data.reset_index(drop=True, inplace=True)
                data["target"] = int(user_id)
                pair_data = pd.concat([pair_data, data], axis=0)
                counter +=1

            #pair_data = pair_data[(pair_data.timestamp1 != pair_data.timestamp2) & (pair_data.site1 != pair_data.site2)]
            pair_data.drop(pair_data[pair_data.duplicated(subset=["site", "timestamp"], keep=False)].index, inplace=True)


            data1 = pair_data[pair_data.target == int(pair[0])][["timestamp", "site"]].copy()
            data2 = pair_data[pair_data.target == int(pair[1])][["timestamp", "site"]].copy()

            if pair[0] not in used:
                data1.to_csv('kaggle_data/new_train/user'+pair[0]+'.csv', index=False)
                used[pair[0]] = 0
            if pair[1] not in used:
                data2.to_csv('kaggle_data/new_train/user'+pair[1]+'.csv', index=False)
                used[pair[1]] = 0                          
    return None

In [417]:
delete_dups_minimal2('kaggle_data/train/*',
                                   feature_names=feature_names, 
                                            site_freq_path="kaggle_data/site_freq.pkl", session_length=10, sort_in_session=True)

In [ ]:
# Find duplicates
def delete_dups_minimal(csv_files_mask, feature_names, site_freq_path="", site_index_path="", dataframe_csv="",
                                    session_length=10, window_size=10, session_time = 30, num_users_for_prediction = 1, sort_in_session=False):
    files = glob(csv_files_mask)
    pair_data = pd.DataFrame()
    
    counter = 1
    
    used = {}
    
    for pair in pairs:
        print pair
        
        if pair[0] not in used or pair[1] not in used:
            files = ['kaggle_data/train_toy/user'+pair[0]+'.csv', 'kaggle_data/train_toy/user'+pair[1]+'.csv']
            counter = 1

            for userfile in files:
                user_id = re.search('user(\d+)\.csv', userfile).group(1)
                data = pd.read_csv(userfile, \
                                   parse_dates=[0], infer_datetime_format=True)
                data.sort_values("timestamp", inplace=True)
                data.reset_index(drop=True, inplace=True)
                data["weekday"] = data.timestamp.apply(lambda x: int(x.isoweekday()))
                data["hour"] = data.timestamp.apply(lambda x: int(x.hour))
                data["target"] = int(user_id)
                print user_id, len(data)
                pair_data = pd.concat([pair_data, data], axis=0)
                counter +=1

            #pair_data = pair_data[(pair_data.timestamp1 != pair_data.timestamp2) & (pair_data.site1 != pair_data.site2)]
            pair_data.drop(pair_data[pair_data.duplicated(subset=["site", "weekday", "hour"], keep=False)].index, inplace=True)
            print len(pair_data)

            data1 = pair_data[pair_data.target == int(pair[0])][["timestamp", "site"]].copy()
            data2 = pair_data[pair_data.target == int(pair[1])][["timestamp", "site"]].copy()

            if pair[0] not in used:
                print 'im here'
                data1.to_csv('kaggle_data/new_train/user'+pair[0]+'.csv', index=False)
                used[pair[0]] = 0
            if pair[1] not in used:
                data2.to_csv('kaggle_data/new_train/user'+pair[1]+'.csv', index=False)
                used[pair[1]] = 0                          

            print used

            break
    return None

In [190]:
delete_dups_minimal('kaggle_data/train_toy/*',
                                   feature_names=feature_names, 
                                            site_freq_path="kaggle_data/site_freq.pkl", session_length=10, sort_in_session=True)

['2631', '786']
2631 357
786 357
4
im here
{'786': 0, '2631': 0}


In [151]:
dataf[(dataf.timestamp1 != dataf.timestamp2) & (dataf.site1 != dataf.site2)]

,timestamp1,site1,timestamp2,site2
9,2014-01-25 10:29:31,fpdownload2.macromedia.com,2014-01-25 10:29:38,www.iegallery.com
10,2014-01-25 10:29:33,go.microsoft.com,2014-01-25 10:29:39,www.iegallery.com
12,2014-01-25 10:29:43,www.iegallery.com,2014-01-25 10:29:47,www.google.fr
13,2014-01-25 10:29:47,www.google.fr,2014-01-25 10:29:48,www.google.com
15,2014-01-25 10:29:48,www.google.fr,2014-01-25 10:30:05,www.google.com
17,2014-01-25 10:30:05,www.google.com,2014-01-25 10:30:10,www.google.fr
19,2014-01-25 10:30:10,ajax.googleapis.com,2014-01-25 10:36:59,office14client.microsoft.com
21,2014-01-25 10:36:59,office14client.microsoft.com,2014-01-25 10:41:10,rr.office.microsoft.com
22,2014-01-25 10:41:10,api.bing.com,2014-01-25 10:41:11,integrate.factiva.com
23,2014-01-25 10:41:10,rr.office.microsoft.com,2014-01-25 10:41:12,www.microsofttranslator.com


In [142]:
dataf

,timestamp1,site1,timestamp2,site2
0,2014-01-17 14:31:24,fpdownload2.macromedia.com,2014-01-17 14:31:24,fpdownload2.macromedia.com
1,2014-01-17 14:31:27,js.microsoft.com,2014-01-17 14:31:27,windows.microsoft.com
2,2014-01-17 14:31:27,go.microsoft.com,2014-01-17 14:31:27,res2.windows.microsoft.com
3,2014-01-17 14:31:27,ajax.microsoft.com,2014-01-17 14:31:27,js.microsoft.com
4,2014-01-17 14:31:31,www.google.com,2014-01-17 14:31:31,www.google.com
5,2014-01-17 14:32:43,ieonline.microsoft.com,2014-01-17 14:32:43,ieonline.microsoft.com
6,2014-01-17 14:36:02,dl.javafx.com,2014-01-17 14:36:02,dl.javafx.com
7,2014-01-17 14:38:34,www.google.fr,2014-01-17 14:38:34,www.google.fr
8,2014-01-17 14:38:48,www.google.fr,2014-01-17 14:38:48,www.google.fr
9,2014-01-25 10:29:31,fpdownload2.macromedia.com,2014-01-25 10:29:38,www.iegallery.com


# Submission

In [144]:
!shuf {folder}train{handler}.vw -o {folder}train{handler}.vw

In [145]:
# Quick
!vw --oaa=550 -d {folder}train{handler}.vw \
-f {folder}initial_model{handler}.model -b 28 -c -k \
--passes=5 -l 0.45 --decay_learning_rate=0.9 \
--keep "b" --keep "c" --keep "a" --keep "z" --keep "s" --cubic "sbc" -q "sb"

!vw -i {folder}initial_model{handler}.model  -t -d {folder}valid{handler}.vw \
-p {folder}vw_valid_pred{handler}.csv --quiet

vw_valid_pred = pd.read_csv(folder+'vw_valid_pred'+handler+'.csv', header=None)
accuracy = accuracy_score(y_valid, vw_valid_pred.values)
print "Accuracy:", accuracy

creating quadratic features for pairs: sb 
creating cubic features for triples: sbc 
using namespaces beginning with: b c a z s 
final_regressor = kaggle_data/initial_model_idf_w8_seq.model
Num weight bits = 28
learning rate = 0.45
initial_t = 0
power_t = 0.5
decay_learning_rate = 0.9
creating cache_file = kaggle_data/train_idf_w8_seq.vw.cache
Reading datafile = kaggle_data/train_idf_w8_seq.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      267        1       21
1.000000 1.000000            2            2.0      538      267       15
1.000000 1.000000            4            4.0      405      214       25
1.000000 1.000000            8            8.0      227      538       27
1.000000 1.000000           16           16.0       98      473       12
1.000000 1.000000           32           32.0      545      214       19
0.9

In [410]:
%%time
!vw --oaa=550 -d {folder}train{handler}.vw \
-f {folder}initial_model{handler}.model -b 28 -c -k \
--passes=30 --decay_learning_rate 0.9 --initial_t 0.002337045080352835 \
-l 0.5416950450219994 \
--power_t 0.5 --loss_function='logistic' --l1 1e-11 --l2 1e-11 \
--cubic="sbc"  \
--keep "s" --keep "b" --keep "c" --keep "l" --keep "a" \
--stage_poly --batch_sz {len(train_file)/6} --batch_sz_no_doubling

creating cubic features for triples: sbc 
using namespaces beginning with: s b c l a 
using l1 regularization = 1e-11
using l2 regularization = 1e-11
final_regressor = kaggle_data/initial_model_idf_w8_pred.model
Num weight bits = 28
learning rate = 0.541695
initial_t = 0.00233705
power_t = 0.5
decay_learning_rate = 0.9
creating cache_file = kaggle_data/train_idf_w8_pred.vw.cache
Reading datafile = kaggle_data/train_idf_w8_pred.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            2            2.0       50      282       33
1.000000 1.000000            4            4.0      367      282       12
1.000000 1.000000            9            9.0      531      282       16
1.000000 1.000000           19           18.9      326      367       18
1.000000 1.000000           39           38.8      386      531       12
1.000000 1.000000           78         

In [411]:
# Prediction on VALID:
!vw -i {folder}initial_model{handler}.model  -t -d {folder}valid{handler}.vw \
-p {folder}vw_valid_pred{handler}.csv --quiet

vw_valid_pred = pd.read_csv(folder+'vw_valid_pred'+handler+'.csv', header=None)
accuracy = accuracy_score(y_valid, vw_valid_pred.values)
print "Accuracy:", accuracy

Accuracy: 0.801177080474


In [146]:
# Prediction on TEST:
!vw -i {folder}initial_model{handler}.model  -t -d {folder}test{handler}.vw \
-p {folder}vw_test_pred{handler}.csv

vw_test_pred = pd.read_csv(folder+'vw_test_pred'+handler+'.csv', header=None)
t_submission = pd.DataFrame(vw_test_pred.astype(int)-1)
vw_subm = class_encoder.inverse_transform(t_submission)

creating quadratic features for pairs: sb 
creating cubic features for triples: sbc 
only testing
predictions = kaggle_data/vw_test_pred_idf_w8_seq.csv
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = kaggle_data/test_idf_w8_seq.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0  unknown      508        6
1.000000 1.000000            2            2.0  unknown      517       46
1.000000 1.000000            4            4.0  unknown      168        6
1.000000 1.000000            8            8.0  unknown       24       27
1.000000 1.000000           16           16.0  unknown      328       19
1.000000 1.000000           32           32.0  unknown      460       25
1.000000 1.000000           64           64.0  unknown      296       29
1.000000 1.000000          128          1

In [147]:
vw_subm

array([['75'],
       ['783'],
       ['1645'],
       ..., 
       ['3027'],
       ['3118'],
       ['1835']], dtype=object)

In [148]:
write_to_submission_file(vw_subm,
             'kaggle_data/49vw_submission_idf_w8_seq.csv')
print "Finished creating submission.\n"

Finished creating submission.



In [29]:
%%time
#9
!vw --oaa=550 -d {folder}train{handler}.vw \
-f {folder}initial_model{handler}.model -b 28 -c -k \
--passes=5 -l 0.45 --decay_learning_rate=0.9 --l1=4e-8 --l2=4e-8 \
-q "sd" -q "sb" --cubic="sbc" --holdout_period 5 --early_terminate 2
#--keep "s" --keep "b" --keep "c" --keep "d" --keep "a"

!vw -i {folder}initial_model{handler}.model  -t -d {folder}test{handler}.vw \
-p {folder}vw_test_pred{handler}.csv --quiet

vw_test_pred = pd.read_csv(folder+'vw_test_pred'+handler+'.csv', header=None)
t_submission = pd.DataFrame(vw_test_pred.astype(int)-1)
vw_subm = class_encoder.inverse_transform(t_submission)


creating quadratic features for pairs: sd sb 
creating cubic features for triples: sbc 
using l1 regularization = 4e-08
using l2 regularization = 4e-08
final_regressor = kaggle_data/initial_model_idf_w8_bal.model
Num weight bits = 28
learning rate = 0.45
initial_t = 0
power_t = 0.5
decay_learning_rate = 0.9
creating cache_file = kaggle_data/train_idf_w8_bal.vw.cache
Reading datafile = kaggle_data/train_idf_w8_bal.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      330        1       33
1.000000 1.000000            2            2.0      180      330       12
1.000000 1.000000            4            4.0      349       54       23
1.000000 1.000000            8            8.0      198      330       25
1.000000 1.000000           16           16.0      548      433       33
1.000000 1.000000           32           32.0      51

In [58]:
vw_pred = pd.read_csv(folder+'vw_test_pred'+handler+'.csv', header=None)
vw_subm = class_encoder.inverse_transform(vw_pred-1)

In [67]:
write_to_submission_file(vw_subm,
             'kaggle_data/31vw_submission_exp.csv')
print "Finished creating submission.\n"

Finished creating submission.



In [65]:
vw_subm = np.copy(vw_subm.astype(float).astype(int).astype(str).astype(object))

In [66]:
vw_subm

array([['2885'],
       ['783'],
       ['2294'],
       ..., 
       ['3027'],
       ['3118'],
       ['2708']], dtype=object)